In [ ]:
import os
from langgraph.graph import StateGraph, MessagesState, START
from typing import Literal

import pandas as pd
from typing import Optional, List, Tuple, Dict, Any
from langchain.docstore.document import Document
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

from paperscraper.pubmed import get_and_dump_pubmed_papers
from paperscraper.utils import load_jsonl
from paperscraper.pdf import save_pdf_from_dump
from IPython.display import display, Image

import utils
import pymupdf
from emulation_utils import *

import re
import json
import pickle
from trialist_utils import *

# for the clinical trial parser
import sys
sys.path.insert(0, os.path.abspath("./trialistUtils"))
from trialistUtils import parse_clinical_trial

# Read variables from .env file into the environment
env_file = '.env'
if os.path.exists(env_file):
    with open(env_file) as f:
        for line in f:
            parts = line.strip().split('=')
            if len(parts) == 2:
                key, value = parts
                print(f"Setting {key} to {value}")
                os.environ[key] = value


from dataframe_utils import Informatician as InformaticianTool
from dataframe_utils import clean_ec

from llm_zoo_rlhf import LLMZoo, GPT4AzureModel, OllamaModel, OpenaiClient, LLMwithReward, LLMwithInteractive
import logging
logging.getLogger("matplotlib").setLevel(logging.WARNING)
logging.getLogger("sentence_transformers").setLevel(logging.WARNING)
logging.getLogger("httpcore").setLevel(logging.CRITICAL)
logging.getLogger("httpx").setLevel(logging.CRITICAL)
logging.getLogger("paperscraper").setLevel(logging.CRITICAL)

import warnings
warnings.filterwarnings("ignore")

set trial id

In [ ]:
# TRIAL_ID = 'NCT04134403'
# TRIAL_ID = 'NCT02856698'
# TRIAL_ID = 'NCT06091982'
# TRIAL_ID = 'NCT00475852'
# TRIAL_ID = 'NCT03872011'
TRIAL_ID = 'NCT04691505'

assert TRIAL_ID in ['NCT04134403', 'NCT02856698', 'NCT06091982', 'NCT00475852', 'NCT03872011', 'NCT04691505'] or TRIAL_ID in ['NCT00000011', 'NCT00000012'], f"TRIAL_ID {TRIAL_ID} is not in the list of valid trial IDs."

set llm

In [ ]:
LLM_MODEL = 'phi4'
# LLM_MODEL = 'deepseek'
# LLM_MODEL = 'gpt4o'
# LLM_MODEL = 'gpt4openai'
# LLM_MODEL = 'gemma3'

assert LLM_MODEL in ['phi4', 'deepseek', 'gpt4o', 'gpt4openai', 'gemma3']
print('llm model:', LLM_MODEL)

In [ ]:
# Defining large language model
# Example usage
llm = LLMZoo()

# openai gpt4 api
openai_api_key = os.getenv('OPENAI_API_KEY')
openai_organization = os.getenv('OPENAI_ORGANIZATION')
openai_project = os.getenv('OPENAI_PROJECT')

# Azure gpt4o api
api_key = os.getenv('AZURE_OPENAI_API_KEY')
endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')
api_version = "2023-05-15"
assert api_key and endpoint and api_key

# Register models with initialization parameters
# llm.register_model("GPT-4o", GPT4AzureModel(api_key=api_key, endpoint=endpoint, api_version=api_version))
# llm.select_model("GPT-4o")

# llm.register_model("OllamaModel", OllamaModel(model='deepseek-r1:14b', temperature=0.0))
llm.register_model('phi4', OllamaModel(model='phi4', temperature=0.0))
llm.register_model('gemma3', OllamaModel(model='gemma3:27b', temperature=0.0))
llm.register_model('deepseek', OllamaModel(model='deepseek-r1:14b', temperature=0.0))
if api_key and endpoint and api_version:
    llm.register_model("gpt4o", GPT4AzureModel(api_key=api_key, endpoint=endpoint, api_version=api_version))
if openai_api_key and openai_organization and openai_project:
    llm.register_model("gpt4openai", OpenaiClient(api_key=openai_api_key, organization=openai_organization, project=openai_project))

llm.select_model(LLM_MODEL)

In [ ]:
extract_pdfs_pubmed = False

In [ ]:
# Semantic search Model initialization
EMBEDDING_MODEL_NAME ='sentence-transformers/all-MiniLM-L6-v2'
encoder = SentenceTransformer(EMBEDDING_MODEL_NAME)

#####################################################

## FOR CLINICIAN AGENT ##
def create_rag_knowledge_base_local(literature_folders, return_docs=False):
    pdf_paths = []
    for literature_folder in literature_folders:
        # Get all PDF files in the folder and its subfolders
        for root, _, files in os.walk(literature_folder):
            for file in files:
                if file.endswith('.pdf'):
                    pdf_paths.append(os.path.join(root, file))

    pdf_strings = []

    for pdf_path in pdf_paths:
        try:
            doc = pymupdf.open(pdf_path)
            keep_pages = None
            remove_pages = None
            pdf_string, _, _, _, _ = utils.create_extraction_components(doc, keep_pages=keep_pages, remove_pages=remove_pages, run_llms=0)
            pdf_strings.append(pdf_string)
        except Exception as e:
            continue

    RAW_KNOWLEDGE_BASE = [
        Document(page_content=pdf_string)
        for pdf_string in pdf_strings
    ]

    # Splitting documents into chunks
    docs_processed = utils.split_documents(
        256,  # We choose a chunk size adapted to our model
        RAW_KNOWLEDGE_BASE,
        tokenizer_name=EMBEDDING_MODEL_NAME,
    )

    docs_content = []
    for doc_processed in docs_processed:
        docs_content.append(doc_processed.page_content)

    # If no documents are found, return an empty DataFrame and index

    if return_docs:
        return docs_content

    if len(docs_content) == 0:
        return pd.DataFrame(), faiss.IndexFlatL2(0)

    # Encoding the documents
    encoder = SentenceTransformer(EMBEDDING_MODEL_NAME)
    vectors = encoder.encode(docs_content)
    df_database = pd.DataFrame(data=docs_content, columns=['text'])
    vector_dimension = vectors.shape[1]
    index = faiss.IndexFlatL2(vector_dimension)
    faiss.normalize_L2(vectors)
    index.add(vectors)
    return df_database, index


def create_rag_knowledge_base_pubmed(question, return_docs=False):
    docs_content = []

    ### Pubmed data
    prompt = f"""
    You are a keyword extraction assistant. The output MUST only be "ANS:" followed by a comma-separated list of keywords. \
    Be concise.

    The keywords should be related to biomedicine and biology. The keywords will be used to parse relevant literature to answer the question.
    Do NOT use keywords related to statistics or data analysis.

    Provide at most 3 keywords from the question below.
    Question: {question}

    Expected Structure Output:
    ANS: keyword1, keyword2, keyword3
    """
    response = llm.invoke(prompt)
    
    if "ANS" in response:
        response = response.split("ANS:")[1].split("\n")[0]
    if "<|eot_id|>" in response:
        response = response.split("<|eot_id|>")[0]
    sep_keywords = response.split(",")
    for i in range(len(sep_keywords)): sep_keywords[i] = sep_keywords[i].strip()
    query_paperscraper = []
    for keyword in sep_keywords: query_paperscraper.append([keyword])
    get_and_dump_pubmed_papers(query_paperscraper, output_filepath='temp/pubmed_result.jsonl')
    abstracts = []
    dates = []
    for paper in load_jsonl('temp/pubmed_result.jsonl'):
        if paper['abstract'] is not None:
            abstracts.append(paper['abstract'])
            dates.append(paper['date'])

    # If no abstracts are found, return an empty DataFrame and index
    if len(abstracts) == 0:
        return pd.DataFrame(), faiss.IndexFlatL2(0)

    # Make RAG_KNOWLEDGE_BASE with all abstracts
    RAW_KNOWLEDGE_BASE = []
    # Sort the abstracts by date, most recent first
    abstracts = [x for _, x in sorted(zip(dates, abstracts), key=lambda pair: pair[0], reverse=True)]
    if len(abstracts) != 0:
        # Get the first 500 abstracts
        abstracts = abstracts[:500]
        for abstract in abstracts: RAW_KNOWLEDGE_BASE.append(Document(page_content=abstract))

        # Splitting documents into chunks
        docs_processed = utils.split_documents(
            256,  # We choose a chunk size adapted to our model
            RAW_KNOWLEDGE_BASE,
            tokenizer_name=EMBEDDING_MODEL_NAME,
        )

        for doc_processed in docs_processed: docs_content.append(doc_processed.page_content)

    if return_docs:
        return docs_content, sep_keywords

    # Encoding the documents
    encoder = SentenceTransformer(EMBEDDING_MODEL_NAME)
    vectors = encoder.encode(docs_content)
    df_database = pd.DataFrame(data=docs_content, columns=['text'])
    vector_dimension = vectors.shape[1]
    index = faiss.IndexFlatL2(vector_dimension)
    faiss.normalize_L2(vectors)
    index.add(vectors)
    return df_database, index, sep_keywords


def get_relevant_info_from_lit(question, df_database_user, index_user):
    # Create the RAG knowledge base
    # df_database, index = create_rag_knowledge_base() ## theoretically this should be done once and then the index should be saved and loaded
    try:
        df_database, index = create_rag_knowledge_base_pubmed(question)
    except:
        pass
    # If the database is empty, return an empty string
    if df_database.empty:
        return "Knowledge base is empty, cannot perform search."
    search_vector = encoder.encode(question)
    _vector = np.array([search_vector])
    faiss.normalize_L2(_vector)
    k = index.ntotal
    distances, ann = index.search(_vector, k=k)
    results = pd.DataFrame({'distances': distances[0], 'ann': ann[0]})
    merge = pd.merge(results, df_database, left_on='ann', right_index=True)
    # Only getting the top 5 results
    combined_texts = ("\n\n").join(list(merge['text'].values[0:5]))
    return combined_texts


def create_rag_knowledge_base_ibkh_triplets(df_ibkh_triplets):
    docs_content = list(df_ibkh_triplets['full_text'].values)

    # If no documents are found, return an empty DataFrame and index
    if len(docs_content) == 0:
        return pd.DataFrame(), faiss.IndexFlatL2(0)

    # Encoding the documents
    encoder = SentenceTransformer(EMBEDDING_MODEL_NAME)
    vectors = encoder.encode(docs_content)
    df_database = pd.DataFrame(data=docs_content, columns=['text'])
    vector_dimension = vectors.shape[1]
    index = faiss.IndexFlatL2(vector_dimension)
    faiss.normalize_L2(vectors)
    index.add(vectors)
    return df_database, index


def get_relevant_info_from_lit_and_pubmed(state, top_k=5):
    """
    Combines knowledge from local files and PubMed to find relevant information.
    
    Args:
        question (str): The user's question.
        top_k (int): The number of top results to return.
        
    Returns:
        str: A combined string of the most relevant text snippets.
    """
    
    question = state['ongoing_question']

    # 1. Get knowledge from PubMed
    pubmed_abstracts_docs, sep_keywords = create_rag_knowledge_base_pubmed(question, return_docs=True)

    # 2. Get knowledge from local files
    if extract_pdfs_pubmed:
        save_pdf_from_dump('temp/pubmed_result.jsonl', pdf_path='Data/Pubmed/.', key_to_save='doi')
        pdf_docs = create_rag_knowledge_base_local(["Data/Sepsis", "Data/Pubmed"], return_docs=True)
        # delete all files in Data/Pubmed
        for filename in os.listdir("Data/Pubmed"):
            file_path = os.path.join("Data/Pubmed", filename)
            if os.path.isfile(file_path):
                os.remove(file_path)
    else:
        pdf_docs = create_rag_knowledge_base_local(["Data/Sepsis"], return_docs=True)

    # remove pubmed_results.jsonl file if it exists
    if os.path.exists('temp/pubmed_result.jsonl'):
        os.remove('temp/pubmed_result.jsonl')

    # 3. Combine the databases and indices
    all_docs_content = []
    if len(pdf_docs) > 0:
        all_docs_content.extend(list(pdf_docs))
    if len(pubmed_abstracts_docs) > 0:
        all_docs_content.extend(list(pubmed_abstracts_docs))
    if not state['variables']['compiled_relevant_texts'].empty:
        all_docs_content.extend(state['variables']['compiled_relevant_texts']['text'].tolist())

    # remove all duplicates from all_docs_content
    all_docs_content = list(set(all_docs_content))

    if len(all_docs_content) == 0:
        return "Knowledge base is empty, cannot perform search."

    # Re-encode and create a new, unified index
    vectors = encoder.encode(all_docs_content)
    combined_df = pd.DataFrame(data=all_docs_content, columns=['text'])
    
    vector_dimension = vectors.shape[1]
    combined_index = faiss.IndexFlatL2(vector_dimension)
    faiss.normalize_L2(vectors)
    combined_index.add(vectors)
    
    # 4. Search the combined index
    search_vector = encoder.encode(question)
    _vector = np.array([search_vector])
    faiss.normalize_L2(_vector)
    
    # Ensure k is not greater than the number of items in the index
    k = min(top_k, combined_index.ntotal)
    if k == 0:
        return "Knowledge base is empty, cannot perform search."

    distances, ann = combined_index.search(_vector, k=k)
    
    results = pd.DataFrame({'distances': distances[0], 'ann': ann[0]})
    merge = pd.merge(results, combined_df, left_on='ann', right_index=True)
    
    # 5. Format and return the results
    combined_texts = "\n\n---\n\n".join(merge['text'].tolist())

    state['variables']['compiled_relevant_texts'] = pd.concat([state['variables']['compiled_relevant_texts'], merge[['text']]], ignore_index=True).reset_index(drop=True)
    state['variables']['compiled_relevant_texts'].drop_duplicates(subset=['text'], inplace=True)

    return combined_texts, state


def get_relevant_info_from_lit_and_pubmed_ibkh(state, top_k=5):
    """
    Combines knowledge from local files and PubMed to find relevant information.
    
    Args:
        question (str): The user's question.
        top_k (int): The number of top results to return.
        
    Returns:
        str: A combined string of the most relevant text snippets.
    """
    
    question = state['ongoing_question']

    # 1. Get knowledge from PubMed
    pubmed_abstracts_docs, sep_keywords = create_rag_knowledge_base_pubmed(question, return_docs=True)

    # 2. Get knowledge from local files
    if extract_pdfs_pubmed:
        save_pdf_from_dump('temp/pubmed_result.jsonl', pdf_path='Data/Pubmed/.', key_to_save='doi')
        pdf_docs = create_rag_knowledge_base_local(["Data/Sepsis", "Data/Pubmed"], return_docs=True)
        # delete all files in Data/Pubmed
        for filename in os.listdir("Data/Pubmed"):
            file_path = os.path.join("Data/Pubmed", filename)
            if os.path.isfile(file_path):
                os.remove(file_path)
    else:
        pdf_docs = create_rag_knowledge_base_local(["Data/Sepsis"], return_docs=True)

    # remove pubmed_results.jsonl file if it exists
    if os.path.exists('temp/pubmed_result.jsonl'):
        os.remove('temp/pubmed_result.jsonl')

    # 3. Combine the databases and indices
    all_docs_content = []
    if len(pdf_docs) > 0:
        all_docs_content.extend(list(pdf_docs))
    if len(pubmed_abstracts_docs) > 0:
        all_docs_content.extend(list(pubmed_abstracts_docs))
    if not df_ibkh_triplets.empty:
        all_docs_content.extend(df_ibkh_triplets['full_text'].tolist())
    if not state['variables']['compiled_relevant_texts'].empty:
        all_docs_content.extend(state['variables']['compiled_relevant_texts']['text'].tolist())

    # remove all duplicates from all_docs_content
    all_docs_content = list(set(all_docs_content))

    if not all_docs_content:
        return "Could not find any relevant information."

    # Re-encode and create a new, unified index
    vectors = encoder.encode(all_docs_content)
    combined_df = pd.DataFrame(data=all_docs_content, columns=['text'])
    
    vector_dimension = vectors.shape[1]
    combined_index = faiss.IndexFlatL2(vector_dimension)
    faiss.normalize_L2(vectors)
    combined_index.add(vectors)
    
    # 4. Search the combined index
    search_vector = encoder.encode(question)
    _vector = np.array([search_vector])
    faiss.normalize_L2(_vector)
    
    # Ensure k is not greater than the number of items in the index
    k = min(top_k, combined_index.ntotal)
    if k == 0:
        return "Knowledge base is empty, cannot perform search."

    distances, ann = combined_index.search(_vector, k=k)
    
    results = pd.DataFrame({'distances': distances[0], 'ann': ann[0]})
    merge = pd.merge(results, combined_df, left_on='ann', right_index=True)
    
    # 5. Format and return the results
    combined_texts = "\n\n---\n\n".join(merge['text'].tolist())

    state['variables']['compiled_relevant_texts'] = pd.concat([state['variables']['compiled_relevant_texts'], merge[['text']]], ignore_index=True).reset_index(drop=True)
    state['variables']['compiled_relevant_texts'].drop_duplicates(subset=['text'], inplace=True)

    return combined_texts, state

#####################################################


## FOR MAIN SUPERVISOR AGENT ##

def get_most_relevant_clinical_trials(user_question):
    # Clean up the user question to only talk about the treatment and disease and other biological information
    prompt = f"""
    You are given a USER QUESTION.

    USER QUESTION: {user_question}

    Your task is to **extract only the core biological content** from the USER QUESTION.

    ### Keep:
    - Treatment (drug, intervention, biomarker, etc.)
    - Disease/condition
    - Outcome (if relevant)

    ### Remove:
    - Any mention of study design, methodology, instructions, protocols, trial setup
    - Generic phrases like "design a trial," "develop a study," or "assess impact"

    Return only the cleaned-up question in the format:

    CLEANED USER QUESTION: <cleaned-up question>

    Example Input / Outpus:
    "What study design should be used to assess the impact of metformin on glucose levels in prediabetic adults?"
    CLEANED USER QUESTION: Does metformin lower glucose in prediabetic adults?

    "Develop an observational study to determine if aspirin reduces stroke risk in elderly patients."
    CLEANED USER QUESTION: Does aspirin reduce stroke risk in elderly patients?

    Do not add any extra words, explanations, or formatting.
    """

    response = llm.invoke(prompt)
    cleaned_user_question = response.split("CLEANED USER QUESTION:")[1].strip()


    # Prompt to get structured keywords of disease and treatment from user question in Essie expression syntax
    prompt = f"""
    You are given a USER QUESTION. 

    USER QUESTION: {cleaned_user_question}

    Your task is to extract the keywords of disease and treatment from the USER QUESTION in Essie expression syntax.

    If there is an outcome, extract the keywords of the outcome as well.

    If there is no outcome, should return OUTCOME: None

    The treatment keywords should not include "treatment" in them.

    Return the keywords in the expected structure output:
    DISEASE: <disease keywords>; TREATMENT: <treatment keywords>; OUTCOME: <outcome keywords>

    Do not provide any additional information or explanation.
    """

    response = llm.invoke(prompt)

    # Parse disease and treatment keywords from the response
    disease_str = response.split("DISEASE: ")[1].split(";")[0].strip()
    trt_str = response.split("TREATMENT: ")[1].split(";")[0].strip()
    disease_keywords = disease_str.split(",")
    trt_keywords = trt_str.split(",")
    outcome_str = response.split("OUTCOME: ")[1].strip()
    outcome_keywords = outcome_str.split(",")
    if outcome_str == "None":
        outcome_keywords = None
    # remove periods from keywords
    disease_keywords = [keyword.replace(".", "") for keyword in disease_keywords]
    trt_keywords = [keyword.replace(".", "") for keyword in trt_keywords]
    essex_disease_str = " AND ".join([f"{keyword.strip()}" for keyword in disease_keywords])
    essex_trt_str = " AND ".join([f"{keyword.strip()}" for keyword in trt_keywords])
    essex_outcome_str = " AND ".join([f"{keyword.strip()}" for keyword in outcome_keywords]) if outcome_keywords else None
    df_clinical = utils.get_clinical_trials_data(essex_disease_str, essex_trt_str, other_keywords_str=essex_outcome_str)


    # Get the most similar trials
    nct_ids = df_clinical['NCT ID'].values.tolist()
    clinical_trials = df_clinical['string'].values.tolist()

    EMBEDDING_MODEL_NAME ='sentence-transformers/all-MiniLM-L6-v2'
    encoder = SentenceTransformer(EMBEDDING_MODEL_NAME)
    docs_content = []
    spliced_nct_ids = []

    RAW_KNOWLEDGE_BASE = []
    if len(clinical_trials) != 0:
        # Get the first 500 abstracts
        for i, clinical_trial in enumerate(clinical_trials):
            RAW_KNOWLEDGE_BASE.append(Document(page_content=clinical_trial, metadata={"source": nct_ids[i]}))

        # Splitting documents into chunks
        docs_processed = utils.split_documents(
            256,  # We choose a chunk size adapted to our model
            RAW_KNOWLEDGE_BASE,
            tokenizer_name=EMBEDDING_MODEL_NAME,
        )

        for doc_processed in docs_processed:
            docs_content.append(doc_processed.page_content)
            spliced_nct_ids.append(doc_processed.metadata["source"])

    # Encoding the documents
    encoder = SentenceTransformer(EMBEDDING_MODEL_NAME)
    vectors = encoder.encode(docs_content)
    df_database = pd.DataFrame(data=docs_content, columns=['text'])
    df_database['NCT ID'] = spliced_nct_ids
    vector_dimension = vectors.shape[1]
    index = faiss.IndexFlatL2(vector_dimension)
    faiss.normalize_L2(vectors)
    index.add(vectors)

    search_vector = encoder.encode(cleaned_user_question)
    _vector = np.array([search_vector])
    faiss.normalize_L2(_vector)
    k = index.ntotal
    distances, ann = index.search(_vector, k=k)
    results = pd.DataFrame({'distances': distances[0], 'ann': ann[0]})
    merge = pd.merge(results, df_database, left_on='ann', right_index=True)

    # Get the first 3 unique NCT IDs
    unique_nct_ids = merge['NCT ID'].unique().tolist()
    # Take the first 3 unique NCT IDs
    closest_nct_ids = unique_nct_ids[:5]
    return closest_nct_ids, closest_nct_ids[0]


#####################################################


## FOR STATISTICIAN AGENT ##
def build_trial_emulation_protocol(state: dict):
    """
    Constructs a structured trial emulation protocol using key information from the state.
    """
    # Extract key trial components from the state for a more focused prompt
    trial_info = {
        "Inclusion Criteria": state['variables'].get('inclusion_criteria', 'Not specified'),
        "Exclusion Criteria": state['variables'].get('exclusion_criteria', 'Not specified'),
        "Treatment Definition": state['variables'].get('treatment_definition', 'Not specified'),
        "Outcome Definition": state['variables']['outcome_definitions'][state['variables']['current_outcome_index']],
    }

    prompt = f"""
    You are a statistician tasked with drafting a clear and structured trial emulation protocol.

    **INSTRUCTIONS:**
    Based on the provided KEY TRIAL COMPONENTS, write a formal trial emulation protocol. The protocol should logically connect these components and briefly outline the analysis plan. It must include the following sections:
    1.  **Objective:** A single sentence stating the goal of the study.
    2.  **Study Population:** Detail the inclusion and exclusion criteria.
    3.  **Intervention and Comparator:** Describe the treatment being emulated.
    4.  **Outcome(s):** Specify the primary and any secondary outcomes.
    5.  **Analysis Plan:** Briefly mention the intended statistical approach (e.g., "A causal inference model will be used to estimate the treatment effect after adjusting for confounders.").

    **KEY TRIAL COMPONENTS:**
    {json.dumps(trial_info, indent=2)}

    **FORMATTING:**
    - Use clear headings for each section.
    - Provide ONLY the protocol text as the answer.

    **ANSWER:**
    """
    state['answering_agents'].append("Statistician")
    state['prompts'].append(prompt)
    response = llm.invoke(prompt, use_cache=False)
    state['responses'].append(response)
    
    print(state['answering_agents'][-1])
    print(state['responses'][-1])
    print("------------------------------------------")
    print("\n\n")

    protocol = response
    return protocol


def select_methods(state, df, trial_protocol, list_of_treatment, list_of_outcome, list_of_duration):
    """
    Selects the best methods based on the dataset and trial protocol using a more structured prompt and robust parsing.
    """
    # Create a summary of key dataset characteristics for the prompt
    dataset_summary = {
        "Number of Patients": df.shape[0],
        "Number of Covariates": df.shape[1],
        "Treatment Column": list_of_treatment[0],
        "Outcome Column": list_of_outcome[0], # Assuming primary outcome is first
        "Duration Column": list_of_duration[0] # Assuming primary duration is first
    }

    prompt = f"""
    You are an expert statistician. Your task is to select the most appropriate statistical methods for a trial emulation study based on the provided protocol and data summary.

    **CONTEXT:**
    - **Trial Emulation Protocol:** {trial_protocol}
    - **Dataset Summary:** {json.dumps(dataset_summary, indent=2)}

    **INSTRUCTIONS:**
    1.  **Select a Covariate Balancing Method** from the following options:
        - Propensity Score Matching (PSM)
        - Inverse Probability of Treatment Weighting (IPTW)
        - No Balancing

    2.  **Select one or more Survival Analysis Methods** from the following options:
        - Cox Proportional Hazards
        - Kaplan-Meier Estimator
        - Parametric Survival Models
        - Random Survival Forest
        - Doubly Robust Estimation

    3.  **Provide a brief, clear rationale** for each choice, considering the data characteristics and research objectives.

    **OUTPUT FORMAT:**
    You MUST structure your response using the following XML-like tags. Do not include any other text outside these tags.
    <BALANCING_METHOD>
        <METHOD>Method Name</METHOD>
        <RATIONALE>Your rationale here</RATIONALE>
    </BALANCING_METHOD>
    <SURVIVAL_ANALYSIS>
        <METHODS>Method1, Method2</METHODS>
        <RATIONALE>Your rationale here</RATIONALE>
    </SURVIVAL_ANALYSIS>

    **ANSWER:**
    """
    state['answering_agents'].append("Statistician")
    state['prompts'].append(prompt)
    response = llm.invoke(prompt, use_cache=False)
    state['responses'].append(response)
    
    print(state['answering_agents'][-1])
    print(state['responses'][-1])
    print("------------------------------------------")
    print("\n\n")

    # Use more robust regex parsing instead of splitting
    try:
        balancing_method = re.search(r"<METHOD>(.*?)</METHOD>", response, re.DOTALL).group(1).strip()
        balancing_rationale = re.search(r"<RATIONALE>(.*?)</RATIONALE>", response, re.DOTALL).group(1).strip()
        
        # Find all rationale blocks and get the second one for survival
        survival_rationale_match = re.findall(r"<RATIONALE>(.*?)</RATIONALE>", response, re.DOTALL)
        survival_rationale = survival_rationale_match[1].strip() if len(survival_rationale_match) > 1 else ""

        survival_methods_str = re.search(r"<METHODS>(.*?)</METHODS>", response, re.DOTALL).group(1).strip()
        survival_methods = [method.strip() for method in survival_methods_str.split(",")]

    except (AttributeError, IndexError) as e:
        balancing_method = "Propensity Score Matching (PSM)"
        balancing_rationale = f"Defaulting to PSM due to parsing error: {e}"
        survival_methods = ["Cox Proportional Hazards"]
        survival_rationale = f"Defaulting to Cox model due to parsing error: {e}"
        
    return balancing_method, balancing_rationale, survival_methods, survival_rationale


def apply_covariate_balancing(state, method, df, categorical_covariates, continuous_covariates, ignore_covariates, list_of_treatment, list_of_outcome, outcome_definition, break_flag, results):
    """
    Applies the selected covariate balancing method to the DataFrame.
    Returns the balanced DataFrame and information about the balancing process.
    """
    scaler = None
    if method == "Propensity Score Matching (PSM)":
        balanced_df, info, results, break_flag, scaler = perform_psm(df, categorical_covariates, continuous_covariates, ignore_covariates, list_of_treatment, list_of_outcome, outcome_definition, break_flag, results, state)
    elif method == "Inverse Probability of Treatment Weighting (IPTW)":
        balanced_df, info, results, break_flag = perform_iptw(df, categorical_covariates, continuous_covariates, ignore_covariates, list_of_treatment, list_of_outcome, outcome_definition, break_flag, results, state)
    elif method == "No Balancing":
        balanced_df, info = df.copy(), "No balancing applied."
    else:
        balanced_df, info = df.copy(), "No balancing applied."
    
    return balanced_df, info, break_flag, results, scaler


def conduct_survival_analysis(state, methods, df, balanced_df, categorical_covariates, continuous_covariates, list_of_treatment, list_of_outcome, list_of_duration, outcome_definition, results, plots):
    """
    Conducts survival analysis using various models.
    Returns the results and generated plots.
    """
    
    str_info = "Survival Analysis Summary:"

    # Semi-parametric Models (Cox Proportional Hazards)
    if "Cox Proportional Hazards" in methods:
        cox_summary, fig, cph_info = run_cox_model(balanced_df, list_of_treatment, list_of_outcome, list_of_duration, outcome_definition, categorical_covariates, continuous_covariates, state)
        results['Cox Proportional Hazards Summary'] = cox_summary
        results['Cox Proportional Hazards Info'] = cph_info
        str_info = str_info + "\n" + cph_info
        plots['Cox Assumption Plots'] = fig
    
    # (Kaplan-Meier Estimator)
    if "Kaplan-Meier Estimator" in methods:
        km_results, km_plot = run_kaplan_meier(balanced_df, list_of_treatment, list_of_outcome, list_of_duration)
        results['Kaplan-Meier Estimator'] = km_results
        plots['Kaplan-Meier Plot'] = km_plot

    # Parametric Models
    if "Parametric Survival Models" in methods:
        parametric_summary, parametric_info = run_parametric_model(balanced_df, list_of_treatment, list_of_outcome, list_of_duration, outcome_definition, state)
        results['Parametric Survival Models Summary'] = parametric_summary
        results['Parametric Survival Models Info'] = parametric_info
        str_info = str_info + "\n" + parametric_info

    # Random Survival Forest
    if "Random Survival Forest" in methods:
        survival_prob_fig, survival_info = run_survival_forest(balanced_df, list_of_treatment, list_of_outcome, list_of_duration, categorical_covariates, continuous_covariates, outcome_definition, state)
        plots['Random Survival Forest Plot'] = survival_prob_fig
        str_info = str_info + "\n" + survival_info

    # Doubly Robust Estimation
    if "Doubly Robust Estimation" in methods:
        dr_results, dr_info = run_doubly_robust(df, list_of_treatment, list_of_outcome, categorical_covariates, continuous_covariates, outcome_definition, state)
        results['Doubly Robust Estimation Results'] = dr_results
        str_info = str_info + "\n" + dr_info
    
    return results, plots, str_info


def compute_SEM(dHRs):
    '''Compute the standard error of the Monte Carlo mean'''
    dHRs = np.array(dHRs)
    SEM = np.mean([np.std(dHRs[:, i])/np.sqrt(dHRs.shape[0]) for i in range(dHRs.shape[1])])
    return SEM


def perform_ec_optmization(state, cache_folder='trials'):
    cache_file = osp.join(cache_folder, f"{state['variables']['trialid']}", 'ec_optimization.pkl')
    if os.path.exists(cache_file):
        with open(cache_file, "rb") as f:
            cached_results = pickle.load(f)
            print("Loaded cached results from file.")
        return cached_results

    all_dataframe = state['variables']['big_dataframe'] 
    ec_info = state['variables']['ec_patients_powerset_dataframe']

    eligibility_list_rule_combos = ec_info['EligibilityCriteria'].tolist()
    eligibility_list_rule_combos = [sorted(combo) for combo in eligibility_list_rule_combos]
    patients_for_each_combo = ec_info['MatchedPatients'].tolist()

    name_rules = np.unique([item for sublist in eligibility_list_rule_combos for item in sublist])
    name_rules = np.sort(name_rules)

    hrs_for_all_combos = []

    # hr dictionary to save per patient_ids_for_combo so that calculations aren't repeated if the same patient_ids_for_combo are used
    previous_patient_ids_for_combo = []

    for i in range(len(eligibility_list_rule_combos)):
        list_of_covariates = copy.deepcopy(state['variables']['Covariates'])
        list_of_treatment = copy.deepcopy(state['variables']['Treatment'])
        list_of_outcome = copy.deepcopy(state['variables']['Outcome'])
        list_of_duration = copy.deepcopy(state['variables']['Duration'])
        patient_column = state['variables']['patient_id']

        # if list_of_treatment is a string, convert it to a list
        if isinstance(list_of_treatment, str):
            list_of_treatment = [list_of_treatment]
        # if list_of_outcome is a string, convert it to a list
        if isinstance(list_of_outcome, str):
            list_of_outcome = [list_of_outcome]
        # if list_of_duration is a string, convert it to a list
        if isinstance(list_of_duration, str):
            list_of_duration = [list_of_duration]

        patient_ids_for_combo = patients_for_each_combo[i]

        # Checking if HR has been calculated for the same patient_ids_for_combo
        continue_flag = False
        for j, previous_patient_ids in enumerate(previous_patient_ids_for_combo):
            if set(previous_patient_ids) == set(patient_ids_for_combo):
                hrs_for_all_combos.append(hrs_for_all_combos[j])
                previous_patient_ids_for_combo.append(patient_ids_for_combo)
                continue_flag = True
                break
        if continue_flag:
            continue

        dataframe = all_dataframe[all_dataframe[patient_column].isin(patient_ids_for_combo)]

        df, categorical_covariates, continuous_covariates, list_of_treatment, list_of_outcome, list_of_duration = clean_data(list_of_covariates,
                                                                                                                            list_of_treatment,
                                                                                                                                list_of_outcome,
                                                                                                                                list_of_duration,
                                                                                                                                    dataframe)

        break_flag = False
        results = {}
        plots = {}

        balancing_method = "Propensity Score Matching"

        # Step 3: Conduct Covariate Balancing
        balanced_df, balancing_str_info, break_flag, results, scaler = apply_covariate_balancing(state, balancing_method, df, categorical_covariates, continuous_covariates, [], list_of_treatment, list_of_outcome, state['variables']['outcome_definition'], break_flag, results)
        if scaler is not None:
            # Inverse transform the balanced dataframe
            balanced_df[continuous_covariates] = scaler.inverse_transform(balanced_df[continuous_covariates])

        # Step 4: Conduct Survival Analysis
        cph, data_cox = calculate_hazard_ratios(balanced_df, list_of_treatment, list_of_outcome, list_of_duration, categorical_covariates, continuous_covariates)
        cox_summary = cph.summary.to_dict()
        hr = cph.summary.loc[list_of_treatment[0]]['exp(coef)']
        hrs_for_all_combos.append(hr)
        previous_patient_ids_for_combo.append(patient_ids_for_combo)

    # Get HR associated with no eligibility criteria
    hr_empty = hrs_for_all_combos[eligibility_list_rule_combos.index([])]
    # Get HR associated with all eligibility criteria, using the longest list of eligibility criteria
    hr_full = hrs_for_all_combos[eligibility_list_rule_combos.index(max(eligibility_list_rule_combos, key=len))]
    dHRs = []

    n_rules = len(name_rules)
    name_rules = np.array(name_rules)
    tolerance = 0.001

    iter_max = 1000
    for m in range(iter_max):
        dHR = np.zeros([n_rules])
        idx = np.random.permutation(n_rules)
        HRs = [hr_empty]
        for i_rule in range(1, n_rules):
            name_rules_subset = name_rules[idx[:i_rule]]
            idx_subset = eligibility_list_rule_combos.index(sorted(list(name_rules_subset)))
            HR = hrs_for_all_combos[idx_subset]
            HRs.append(HR)
        HRs.append(hr_full)
        dHR[idx] = np.array([HRs[i]-HRs[i-1] for i in range(1, len(HRs))])
        dHRs.append(dHR)
        # Convergence checking
        SEM = compute_SEM(dHRs)
        print('Shapley Computation Iteration %d | SEM = %.4f' % (m, SEM))
        if (m>0) and (SEM < tolerance):
            print('Stopping criteria satisfied!')
            break
    if m == (iter_max-1):
        print('Maximum iteration reached!')
    shapley_value = np.mean(dHRs, axis=0)

    with open(cache_file, "wb") as f:
        pickle.dump((shapley_value, name_rules, hrs_for_all_combos), f)
        print(f"Results saved to {cache_file}.")

    return shapley_value, name_rules, hrs_for_all_combos


def reask_subgroup_to_clinician(state, continuous_covariates_lower, categorical_covariates_lower, strat_covariate, list_of_outcome, balanced_df_subgroup):
    subgroup_analyses = ""
    for key, value in state['variables'][list_of_outcome[0]]['subgroup_analysis_dict'].items():
        subgroup_analyses += f"{key}\n{value}\n\n"

    # Results are not significant so we want to ask the clinican to ask for a stratification on a covariate
    # Provide all the continuous covariates and their ranges
    # Provide all the categorical covariates and their unique values
    continuous_covariates_records = {}
    for col in continuous_covariates_lower:
        continuous_covariates_records[col] = [balanced_df_subgroup[col].min(), balanced_df_subgroup[col].max()]
    categorical_covariates_records = {}
    for col in categorical_covariates_lower:
        categorical_covariates_records[col] = balanced_df_subgroup[col].unique().tolist()
    issue = f'''
    Clinician, the current hazard ratio is not significant for any subgroup analyses. Please provide a covariate to stratify on. Also provide the level to stratify on.

    You are given the following covariates and their ranges:

    Continuous Covariates:
    {continuous_covariates_records}

    Categorical Covariates:
    {categorical_covariates_records}

    You are also given previous stratification results:
    {subgroup_analyses}

    Do **NOT** stratify on this covariate: {strat_covariate}
    
    Expected Structure Output:
    STRATIFY ON: <Covariate>
    LEVEL: <Level>

    Provide a **DIFFERENT** covariate to stratify on. It should not be in the previous stratification results or {strat_covariate}.
    The Covariate should be one of the covariates provided above. It should be spelled and use the exact same capitalization as in the list.
    Level should be a value within the range of the covariate. \
    Level **MUST** be a number. Do not output a placeholder or a string. It will be parsed and used for stratification.

    Do not provide additional information or context in the response. No explanation is needed. Provide BOTH "STRATIFY ON" and "LEVEL" in the response.
    ONLY provide **ONE** stratification.
    '''
    state['recent_response'] = issue
    state['sending_agent'] = "Statistician"
    state['predefined_next_agent'] = "Clinician"
    state['predefined_next_question'] = issue
    state['next'] = "__end__"

    return state


def _handle_subgroup_analysis(state, list_of_outcome, list_of_duration):
    """
    Handles the subgroup analysis based on the ongoing question and the current state of the variables.
    state: The current state of the system.
    """
    list_of_outcomes = state['variables']['Outcome']
    list_of_outcomes = [outcome.lower() for outcome in list_of_outcomes]

    if 'number_time_entered_subgroup_analysis' not in state['variables'][list_of_outcome[0]]:
        state['variables'][list_of_outcome[0]]['number_time_entered_subgroup_analysis'] = 0

    state['variables'][list_of_outcome[0]]['number_time_entered_subgroup_analysis'] += 1

    subgroup_results_str = ""
    balanced_df_subgroup = state['variables'][list_of_outcome[0]]['balanced_df'].copy()
    continuous_covariates = state['variables'][list_of_outcome[0]]['continuous_covariates']
    categorical_covariates = state['variables'][list_of_outcome[0]]['categorical_covariates']
    list_of_covariates = state['variables'][list_of_outcome[0]]['categorical_covariates'] + state['variables'][list_of_outcome[0]]['continuous_covariates']
    list_of_treatment = state['variables']['Treatment']
    response = state['ongoing_question']

    if state['variables'][list_of_outcome[0]]['number_time_entered_subgroup_analysis'] <= 5:
        # --------------------------
        # Step 1: Load and Prepare Data
        # --------------------------
        ## 1.1 stratified variables
        # get the last substring with "STRATIFY ON:"
        response = "STRATIFY ON:" + response.split("STRATIFY ON:")[-1]

        strat_covariate = re.split(r'[\n,]', response.split("STRATIFY ON:")[1])[0].strip()

        # if list_of_treatment is a string, convert it to a list
        if isinstance(list_of_treatment, str):
            list_of_treatment = [list_of_treatment]
        # if list_of_outcome is a string, convert it to a list
        if isinstance(list_of_outcome, str):
            list_of_outcome = [list_of_outcome]
        # if list_of_duration is a string, convert it to a list
        if isinstance(list_of_duration, str):
            list_of_duration = [list_of_duration]
        list_of_covariates_lower = [col.lower() for col in list_of_covariates]
        continuous_covariates_lower = [col.lower() for col in continuous_covariates]
        categorical_covariates_lower = [col.lower() for col in categorical_covariates]
        
        # Make all columns in balanced_df_subgroup which are in list_of_covariates_lower lowercase
        new_cols = []
        for col in balanced_df_subgroup.columns:
            if (col.lower() in continuous_covariates_lower) or (col.lower() in categorical_covariates_lower):
                new_cols.append(col.lower())
            else:
                new_cols.append(col)
        balanced_df_subgroup.columns = new_cols

        ## 1.2 stratified values(levels)
        # level_strat = response.split("LEVEL:")[1].split("\n|,")[0].strip()
        level_strat = re.split(r'[\n,]', response.split("LEVEL:")[1])[0].strip()
        # if >, <, >=, <= is in the level_strat, then convert it to the correct format
        if ">=" in level_strat:
            level_strat = level_strat.replace(">=", "").strip()
        elif "<=" in level_strat:
            level_strat = level_strat.replace("<=", "").strip()
        elif ">" in level_strat:
            level_strat = level_strat.replace(">", "").strip()
        elif "<" in level_strat:
            level_strat = level_strat.replace("<", "").strip()

        if '`' in level_strat:
            level_strat = level_strat.replace("`", "").strip()
        if '%' in level_strat:
            level_strat = level_strat.replace("%", "").strip()

        
        # Check if level_strat can be converted to a float
        try:
            level_strat = float(level_strat.strip())
        except:
            return reask_subgroup_to_clinician(state, continuous_covariates_lower, categorical_covariates_lower, strat_covariate, list_of_outcome, balanced_df_subgroup)


        # make strat_covariate lowercase
        strat_covariate = strat_covariate.lower()

        # ask llm to make sure strat_covariate is in all_covariates_lower and if not find the closest match
        all_covariates_lower = continuous_covariates_lower + categorical_covariates_lower
        if strat_covariate not in all_covariates_lower:
            llm_prompt = f"""
            You are a statistician. The covariate to stratify on is not in the list of covariates. Please provide the closest match to the covariate to stratify on.

            The covariate to stratify on is: {strat_covariate}

            The available covariates are: {all_covariates_lower}

            The match **MUST** be one of the covariates provided above. It **SHOULD** be spelled the **EXACT** same way.

            Do not provide additional information or context in the response. No explanation is needed.

            Expected Structure Output:
            STRATIFY ON: <Covariate>
            """
            
            state['answering_agents'].append("Statistician")
            state['prompts'].append(llm_prompt)
            response_llm = llm.invoke(llm_prompt, use_cache=False)
            state['responses'].append(response_llm)

            print(state['answering_agents'][-1])
            print(state['responses'][-1])
            print("------------------------------------------")
            print("\n\n")

            if "STRATIFY ON" in response_llm:
                strat_covariate = response_llm.split("STRATIFY ON:")[1].split("\n|,")[0].strip().lower()


        key = f"Stratifying by {strat_covariate} at {level_strat}, Subgroup A: {strat_covariate} < {level_strat}, Subgroup B: {strat_covariate} >= {level_strat}"

        # Check if strat_covariate is in balanced_df_subgroup as a final check
        if strat_covariate not in balanced_df_subgroup.columns:
            return reask_subgroup_to_clinician(state, continuous_covariates_lower, categorical_covariates_lower, strat_covariate, list_of_outcome, balanced_df_subgroup)


        # check if level_strat is within the range of the strat_covariate
        if balanced_df_subgroup[strat_covariate].min() > level_strat or balanced_df_subgroup[strat_covariate].max() < level_strat:
            return reask_subgroup_to_clinician(state, continuous_covariates_lower, categorical_covariates_lower, strat_covariate, list_of_outcome, balanced_df_subgroup)

                    
        balanced_df_subgroup['strat_group'] = np.where(balanced_df_subgroup[strat_covariate] < level_strat, 'SubgroupA', 'SubgroupB')
        balanced_df_subgroup[f"{list_of_treatment[0]}_{strat_covariate}"] = balanced_df_subgroup[list_of_treatment[0]] * balanced_df_subgroup['strat_group'].map({'SubgroupA': 0, 'SubgroupB': 1})

        # --------------------------
        # Step 2: Fit Cox PH Model
        # --------------------------
        cph = lifelines.fitters.coxph_fitter.CoxPHFitter(penalizer=0.1)
        cph_formula = f"{list_of_treatment[0]} + strat_group + {list_of_treatment[0]}_{strat_covariate}"
        # Add all the other covariates to the formula
        for covariate in list_of_covariates_lower:
            if covariate not in [list_of_treatment[0], strat_covariate]:
                cph_formula += f" + {covariate}"

        try:
            if "iptw_weight" in list(balanced_df_subgroup):
                cph.fit(
                    balanced_df_subgroup,
                    duration_col=list_of_duration[0],
                    event_col=list_of_outcome[0],
                    robust=True,
                    weights_col="iptw_weight",
                    formula=cph_formula)

            else:
                cph.fit(
                    balanced_df_subgroup,
                    duration_col=list_of_duration[0],
                    event_col=list_of_outcome[0],
                    robust=True,
                    formula=cph_formula
                )
        except:
            return reask_subgroup_to_clinician(state, continuous_covariates_lower, categorical_covariates_lower, strat_covariate, list_of_outcome, balanced_df_subgroup)


        # --------------------------
        # Step 3: Extract Coefficients and Covariance Matrix
        # --------------------------
        params = cph.params_
        cov_matrix = cph.variance_matrix_

        # --------------------------
        # Step 4: Calculate HRs
        # --------------------------
        beta_treatment = params[list_of_treatment[0]]
        beta_interaction = params[f"{list_of_treatment[0]}_{strat_covariate}"]
        hr_subgroupA = np.exp(beta_treatment)
        hr_subgroupB = np.exp(beta_treatment + beta_interaction)

        # --------------------------
        # Step 5: Calculate CIs
        # --------------------------
        se_treatment = cph.standard_errors_[list_of_treatment[0]]
        se_interaction = cph.standard_errors_[f"{list_of_treatment[0]}_{strat_covariate}"]
        cov_treatment_interaction = cov_matrix.loc[list_of_treatment[0], f"{list_of_treatment[0]}_{strat_covariate}"]
        se_subgroupB = np.sqrt(cov_matrix.loc[list_of_treatment[0], list_of_treatment[0]] +
                            cov_matrix.loc[f"{list_of_treatment[0]}_{strat_covariate}", f"{list_of_treatment[0]}_{strat_covariate}"] +
                            2 * cov_matrix.loc[list_of_treatment[0], f"{list_of_treatment[0]}_{strat_covariate}"])
        ci_subgroupA_lower = np.exp(beta_treatment - 1.96 * se_treatment)
        ci_subgroupA_upper = np.exp(beta_treatment + 1.96 * se_treatment)
        ci_subgroupB_lower = np.exp(beta_treatment + beta_interaction - 1.96 * se_subgroupB)
        ci_subgroupB_upper = np.exp(beta_treatment + beta_interaction + 1.96 * se_subgroupB)

        # --------------------------
        # Step 6: Calculate p-values
        # --------------------------
        p_subgroupA = cph.summary.loc[list_of_treatment[0], 'p']
        z_subgroupB = (beta_treatment + beta_interaction) / se_subgroupB
        p_subgroupB = 2 * (1 - scipy.stats.norm.cdf(abs(z_subgroupB)))

        any_significance = False

        subgroup_results_str += f"HR for Treatment vs. Control for Subgroup A: {hr_subgroupA:.4f}\n"
        subgroup_results_str += f"95% CI for HR in Subgroup A: ({ci_subgroupA_lower:.4f}, {ci_subgroupA_upper:.4f})\n"
        subgroup_results_str += f"p-value for HR in Subgroup A: {p_subgroupA:.4f}\n"

        if p_subgroupA < 0.05:
            any_significance = True
            subgroup_results_str += f"Subgroup A is statistically significant.\n\n"
        else:
            subgroup_results_str += f"Subgroup A is NOT statistically significant.\n\n"

        subgroup_results_str += f"HR for Treatment vs. Control for Subgroup B: {hr_subgroupB:.4f}\n"
        subgroup_results_str += f"95% CI for HR in Subgroup B: ({ci_subgroupB_lower:.4f}, {ci_subgroupB_upper:.4f})\n"
        subgroup_results_str += f"p-value for HR in Subgroup B: {p_subgroupB:.4f}\n"

        if p_subgroupB < 0.05:
            any_significance = True
            subgroup_results_str += f"Subgroup B is statistically significant.\n\n"
        else:
            subgroup_results_str += f"Subgroup B is NOT statistically significant.\n\n"

        state['variables'][list_of_outcome[0]]['subgroup_analysis_dict'][key] = subgroup_results_str

        if not any_significance and len(state['variables'][list_of_outcome[0]]['subgroup_analysis_dict']) < 5:
            return reask_subgroup_to_clinician(state, continuous_covariates_lower, categorical_covariates_lower, strat_covariate, list_of_outcome, balanced_df_subgroup)

    state['ongoing_question'] = ""
    state['current_information'] = f"Finished analysis for {list_of_outcome[0]}."
    state['variables']['completed_outcomes'].append(list_of_outcome[0])
    if len(list_of_outcomes) > (state['variables']['current_outcome_index'] + 1):
        state['ongoing_question'] = f"Please proceed to the next outcome: {list_of_outcomes[state['variables']['current_outcome_index'] + 1]}."
        state['current_information'] += f" Proceeding to the next outcome: {list_of_outcomes[state['variables']['current_outcome_index'] + 1]}."
    state['variables']['current_outcome_index'] += 1
    state['next'] = "Statistician"

    return state


def _parse_covariate_modifications(state, dataframe, list_of_outcome, llm):
    most_recent_response = state['ongoing_question'] 
    llm_prompt = f"""
    You are a statistician. Based on the most recent response, determine if there are any covariates to drop or add.
    You might see a response like the following:

    body mass index (BMI)|DROP|N/A|Less critical for survival analysis in sepsis patients compared to other factors.
    creatinine|SUBSTITUTE|blood urea nitrogen (BUN)|BUN can reflect kidney function similarly to creatinine.
    heart rate|SUBSTITUTE|mean arterial pressure (MAP)|MAP indirectly reflects cardiovascular function and is more consistently available.

    In this case, the covariates to drop are 'body mass index (BMI)', 'creatinine', and 'heart rate'
    The covariates to add are 'blood urea nitrogen (BUN)' and 'mean arterial pressure (MAP)'

    Sometimes, you might decide to not drop or add covariates, but rather just ignore it. This might be due to \
    the fact that the the covariate cannot be substituted or dropped or post-processing after balancing is the recommended approach.
    In that case, add this to ignore covariates.

    Sometimes the most recent response does not have to do anything with covariates, \
    in that case, output 'No covariates to drop' or 'No covariates to add'.

    The most recent response is as follows:
    {most_recent_response}

    If there are any covariates to drop, provide a list of the covariates to be dropped in the expected output structure:
    DROP COVARIATES: <Covariate1>,<Covariate2>, ...
    ADD COVARIATES: <Covariate1>,<Covariate2>, ...
    IGNORE COVARIATES: <Covariate1>,<Covariate2>, ...

    If no covariates are to be dropped, output 'No covariates to drop'. Do not output 'DROP COVARIATES: '.
    If no covariates are to be added, output 'No covariates to add'. Do not output 'ADD COVARIATES: '.

    Do not additional information or context in the response. No explanations or extra formatting that what is described above for the output structure.

    ANSWER:
    """

    state['answering_agents'].append("Statistician")
    state['prompts'].append(llm_prompt)
    response_llm_drop = llm.invoke(llm_prompt, use_cache=False)
    state['responses'].append(response_llm_drop)
    
    print(state['answering_agents'][-1])
    print(state['responses'][-1])
    print("------------------------------------------")
    print("\n\n")

    covariates_to_drop = []
    covariates_to_add = []
    covariates_to_ignore = []
    try:
        if "No covariates to drop" in response_llm_drop:
            covariates_to_drop = []
        if "No covariates to add" in response_llm_drop:
            covariates_to_add = []
        if "DROP COVARIATES" in response_llm_drop:
            covariates_to_drop = response_llm_drop.split("DROP COVARIATES:")[1].strip().split(",")
            # strip each covariate and convert to lower case
            covariates_to_drop = [covariate.strip().lower() for covariate in covariates_to_drop]
        if "ADD COVARIATES" in response_llm_drop:
            covariates_to_add = response_llm_drop.split("ADD COVARIATES:")[1].strip().split(",")
            # strip each covariate and convert to lower case
            covariates_to_add = [covariate.strip().lower() for covariate in covariates_to_add]
        if "IGNORE COVARIATES" in response_llm_drop:
            covariates_to_ignore = response_llm_drop.split("IGNORE COVARIATES:")[1].strip().split(",")
            # strip each covariate and convert to lower case
            covariates_to_ignore = [covariate.strip().lower() for covariate in covariates_to_ignore]
    except:
        pass

    # See if there are columns with only NaN values
    columns_with_nan = dataframe.columns[dataframe.isna().all()].tolist()
    # Add these columns to the drop covariates
    covariates_to_drop.extend(columns_with_nan)

    state['variables'][list_of_outcome[0]]['ignore_covariates'].extend(covariates_to_ignore)
    state['variables'][list_of_outcome[0]]['add_covariates'].extend(covariates_to_add)
    state['variables'][list_of_outcome[0]]['drop_covariates'].extend(covariates_to_drop)

    # Remove any covariates in covariates_to_add that are in covariates_to_drop
    state['variables'][list_of_outcome[0]]['add_covariates'] = [covariate for covariate in state['variables'][list_of_outcome[0]]['add_covariates'] if covariate not in state['variables'][list_of_outcome[0]]['drop_covariates']]

    # make sure there are no duplicates in ignore_covariates, add_covariates, and drop_covariates
    state['variables'][list_of_outcome[0]]['ignore_covariates'] = list(set(state['variables'][list_of_outcome[0]]['ignore_covariates']))
    state['variables'][list_of_outcome[0]]['add_covariates'] = list(set(state['variables'][list_of_outcome[0]]['add_covariates']))
    state['variables'][list_of_outcome[0]]['drop_covariates'] = list(set(state['variables'][list_of_outcome[0]]['drop_covariates']))

    covariates_to_add = state['variables'][list_of_outcome[0]]['add_covariates']
    covariates_to_drop = state['variables'][list_of_outcome[0]]['drop_covariates']

    return state, covariates_to_add, covariates_to_drop, covariates_to_ignore


def _generate_final_report(state: dict, llm: LLMZoo) -> dict:
    """
    Assembles all analysis results, calculates sample size, and uses an LLM
    to generate a final, structured report.
    """
    # --- 1. GATHER AND STRUCTURE RESULTS ---
    outcomes = [o.lower() for o in state['variables'].get('Outcome', [])]
    outcome_defs = state['variables'].get('outcome_definitions', [])
    
    # Primary Outcome
    primary_outcome_col = outcomes[0]
    primary_outcome_def = outcome_defs[0]
    primary_results_data = state['variables'].get(primary_outcome_col, {})
    primary_analysis_str = primary_results_data.get('subgroup_analysis_dict', {}).get('No Stratification', 'Not performed.')

    # Trial Protocol
    trial_protocol = state['variables'][primary_outcome_col].get('trial_protocol', 'No trial protocol provided.')

    # Balancing Information for Primary Outcome
    balancing_info = state['variables'][primary_outcome_col].get('balancing_information', 'No balancing information provided.')
    
    # Primary Subgroup Analyses
    primary_subgroup_list = []
    for key, value in primary_results_data.get('subgroup_analysis_dict', {}).items():
        if key != 'No Stratification':
            primary_subgroup_list.append(value)
    primary_subgroup_str = "\n\n".join(primary_subgroup_list) if primary_subgroup_list else "No significant subgroups were found or explored."

    # Secondary/Adverse Event Outcomes
    secondary_results_list = []
    if len(outcomes) > 1:
        for i, outcome_col in enumerate(outcomes[1:]):
            outcome_data = state['variables'].get(outcome_col, {})
            analysis_summary = outcome_data.get('subgroup_analysis_dict', {}).get('No Stratification', 'Not performed.')
            secondary_results_list.append(f"--- Outcome: {outcome_defs[i+1]} ---\n{analysis_summary}")
    secondary_results_str = "\n\n".join(secondary_results_list) if secondary_results_list else "No secondary outcomes were analyzed."

    # --- 2. PERFORM SAMPLE SIZE CALCULATION ---
    sample_size_str = "Sample size calculation was not performed."
    primary_cox_summary = primary_results_data.get('results', {}).get('Cox Proportional Hazards Summary')
    if primary_cox_summary is not None:
        try:
            treatment_col = state['variables']['Treatment']
            if isinstance(treatment_col, list): treatment_col = treatment_col[0]
            
            hr = primary_cox_summary['exp(coef)'][treatment_col]
            size, _ = compute_cox_sample_size(
                primary_results_data['balanced_df'],
                treatment_col,
                primary_outcome_col,
                state['variables']['Duration'][0].lower(),
                hr
            )
            sample_size_str = f"Based on the observed data and effect size, the estimated required sample size to achieve 80% power at an alpha of 0.05 is {size} patients."
        except Exception as e:
            sample_size_str = f"Could not perform sample size calculation due to an error: {e}"

    # -- E-Value Calculation --
    evalue_str = "E-Value calculation was not performed."
    primary_cox_summary = primary_results_data.get('results', {}).get('Cox Proportional Hazards Summary')
    if primary_cox_summary is not None:
        try:
            hr = primary_cox_summary['exp(coef)'][treatment_col]
            evalue_results = calculate_evalues(
                primary_results_data['balanced_df'],
                primary_outcome_col,
                hr
            )
            evalue = evalue_results["e_values"]["point_estimate"]
            evalue_str = f"The minimum strength of association that an unmeasured confounder would need to have with both the exposure and the outcome to completey eliminate the observed effect is {evalue}. "
        except Exception as e:
            evalue_str = f"Could not perform E-Value calculation due to an error."

    
    # --- 3. CONSTRUCT THE IMPROVED LLM PROMPT ---
    report_context = f"""
    ### INITIAL QUESTION
    {state["initial_question"]}

    ### PRIMARY OUTCOME: {primary_outcome_def}
    #### Trial Protocol
    {trial_protocol}

    #### Balancing Information
    {balancing_info}

    #### Main Analysis Results
    {primary_analysis_str}

    #### Subgroup Analysis Results
    {primary_subgroup_str}

    ### SECONDARY / ADVERSE EVENT OUTCOMES
    {secondary_results_str}

    ### SAMPLE SIZE CALCULATION
    {sample_size_str}

    ### E-VALUE CALCULATION
    {evalue_str}
    """

    final_report_prompt = f"""
    You are an expert medical writer and statistician tasked with composing a formal, publication-quality report based on the results of a trial emulation study.

    **CONTEXT AND DATA:**
    You are provided with the initial research question and a structured summary of all statistical analyses performed.
    ---
    {report_context}
    ---

    **INSTRUCTIONS:**
    Based ONLY on the provided context and data, generate a complete and well-structured report. The report MUST include all of the following sections, in this order:

    1.  **Title Page:** A clear and informative title for the study.
    2.  **Abstract:** A concise summary covering the study's objective, methods, key results (for primary and secondary outcomes), and conclusions.
    3.  **Introduction:** Provide background on the clinical question, the rationale for the trial emulation, and the study's objectives.
    4.  **Methods:** Briefly describe the data sources, key eligibility criteria, treatment strategies, and the statistical methods used (e.g., confounder adjustment, outcome modeling). Also mention biases that were addressed.
    5.  **Results:** Present the main findings.
        * Start with the primary outcome results.
        * Incorporate key numerical results, such as Hazard Ratios (HR), 95% Confidence Intervals (CI), and p-values where provided.
        * Include the results from any subgroup analyses that were performed.
        * Detail the findings for all secondary/adverse event outcomes.
    6.  **Discussion:** Interpret the key findings. Discuss the consistency of the results, the study's strengths, and its limitations (e.g., if the sample size was noted as a concern). Mention implications for future research.
    7.  **Conclusion:** Briefly summarize the study's main takeaways and contributions.
    8.  **References:** Add a placeholder section for references.

    **WRITING STYLE:**
    * Be objective, formal, and use clear scientific language.
    * When reporting numerical results, explain their meaning in the context of the study (e.g., "The hazard ratio of 1.10 suggests a 10% increased risk...").
    * Pay close attention to confidence intervals and p-values when making statements about statistical significance.

    Generate the complete report. Provide ONLY the report and NOTHING else.
    """

    # --- 4. CALL LLM AND UPDATE STATE ---
    final_report = llm.invoke(final_report_prompt, use_cache=False)
    
    # This logging was in the original code, it's good practice to keep it
    state['answering_agents'].append("Statistician")
    state['prompts'].append(final_report_prompt)
    state['responses'].append(final_report)
    
    print("--- Agent: Statistician ---")
    print(final_report)
    print("------------------------------------------\n\n")

    report = final_report.split("<|eot_id|>")[0].strip()
    state['current_report'] = report
    state['recent_response'] = report + "\n\nPlease review the report and provide feedback."
    state['sending_agent'] = "Statistician"
    state['predefined_next_agent'] = "Reviewer"
    state['next'] = "__end__"
    
    print("Generated Report by Statistician")

    return state


In [ ]:
# Define the state class for the agents
class TopAgentState(MessagesState):
    next: Literal["MainSupervisor", "ClinicianAgentInvoker", "InformaticianAgentInvoker", "TrialistInvoker", "__end__"]
    initial_question: str = ""  # Initial user question
    predefined_next_agent: str = ""  # Predefined next agent
    predefined_next_question: str = ""  # Predefined next question
    sending_agent: str = ""  # Sending agent, agent who sent the most recent response
    ongoing_question: str = ""  # Ongoing question
    recent_response: str = ""  # Most recent response from agent
    current_information: str = ""  # All collected information, minimized for entry into the next agent
    all_information: str = ""  # All collected information
    all_information_index: None  # Index for the faiss index
    all_information_database: None  # Database for the faiss index
    current_report: str = ""  # Current report
    final_summary: str = ""  # Final summary
    prompts: List[str] = []
    responses: List[str] = []
    answering_agents: List[str] = []
    # Statistician specific variables
    variables: Dict[str, Any] = {}

class InformaticianState(MessagesState):
    next: Literal["Informatician", "__end__"]
    initial_question: str = ""  # Initial user question
    predefined_next_agent: str = ""  # Predefined next agent
    predefined_next_question: str = ""  # Predefined next question
    sending_agent: str = ""  # Sending agent, agent who sent the most recent response
    ongoing_question: str = ""  # Ongoing question
    recent_response: str = ""  # Most recent response from agent
    current_information: str = ""  # All collected information, minimized for entry into the next agent
    all_information: str = ""  # All collected information
    all_information_index: None  # Index for the faiss index
    all_information_database: None  # Database for the faiss index
    current_report: str = ""  # Current report
    final_summary: str = ""  # Final summary
    prompts: List[str] = []
    responses: List[str] = []
    answering_agents: List[str] = []
    # Informatician specific variables
    variables: Dict[str, Any] = {}

class StatisticianState(MessagesState):
    next: Literal["Statistician", "__end__"]
    initial_question: str = ""  # Initial user question
    predefined_next_agent: str = ""  # Predefined next agent
    predefined_next_question: str = ""  # Predefined next question
    sending_agent: str = ""  # Sending agent, agent who sent the most recent response
    ongoing_question: str = ""  # Ongoing question
    recent_response: str = ""  # Most recent response from agent
    current_information: str = ""  # All collected information, minimized for entry into the next agent
    all_information: str = ""  # All collected information
    all_information_index: None  # Index for the faiss index
    all_information_database: None  # Database for the faiss index
    current_report: str = ""  # Current report
    final_summary: str = ""  # Final summary
    prompts: List[str] = []
    responses: List[str] = []
    answering_agents: List[str] = []
    # Statistician specific variables
    variables: Dict[str, Any] = {}

class ClinicianState(MessagesState):
    next: Literal["Clinician" "__end__"]
    initial_question: str = ""  # Initial user question
    predefined_next_agent: str = ""  # Predefined next agent
    predefined_next_question: str = ""  # Predefined next question
    sending_agent: str = ""  # Sending agent, agent who sent the most recent response
    ongoing_question: str = ""  # Ongoing question
    recent_response: str = ""  # Most recent response from agent
    current_information: str = ""  # All collected information, minimized for entry into the next agent
    all_information: str = ""  # All collected information
    all_information_index: None  # Index for the faiss index
    all_information_database: None  # Database for the faiss index
    current_report: str = ""  # Current report
    final_summary: str = ""  # Final summary
    prompts: List[str] = []
    responses: List[str] = []
    answering_agents: List[str] = []
    variables: Dict[str, Any] = {}  # Clinician specific variables

class TrialistState(MessagesState):
    # Haoyang, add variables as you need, right now it's only global variables
    next: Literal["Trialist", "__end__"]
    initial_question: str = ""  # Initial user question
    predefined_next_agent: str = ""  # Predefined next agent
    predefined_next_question: str = ""  # Predefined next question
    sending_agent: str = ""  # Sending agent, agent who sent the most recent response
    ongoing_question: str = ""  # Ongoing question
    recent_response: str = ""  # Most recent response from agent
    current_information: str = ""  # All collected information, minimized for entry into the next agent
    all_information: str = ""  # All collected information
    all_information_index: None  # Index for the faiss index
    all_information_database: None  # Database for the faiss index
    current_report: str = ""  # Current report
    final_summary: str = ""  # Final summary
    prompts: List[str] = []
    responses: List[str] = []
    answering_agents: List[str] = []
    # Trialist specific variables
    variables: Dict[str, Any] = {}

In [ ]:
def MainSupervisor(state: TopAgentState):
    """
    Main supervisor determines which top-level agent to delegate the task to.
    Currently contains temporary code to delegate tasks dynamically.
    """

    # Add ongoing question for context of most recent response, if question exists
    if state['ongoing_question'] != "":  # fix bug
        state['current_information'] = state['current_information'] + state['ongoing_question']

    if state["recent_response"] != "":
        state["current_information"] = state["current_information"] + state["recent_response"]
    
    # Summarize current_information if it is larger than 100000 tokens
    if len(state["current_information"].split()) > 100000:
        state['all_information'] = state['all_information'] + state['current_information']
        # Use an LLM to summarize the current_information into something smaller, keeping the most relevant details for the initial_question
        prompt = f"""
        You are an expert in summarizing information. Summarize the CURRENT INFORMATION into a smaller summary.
        Keep the most relevant details for the INITIAL QUESTION.
        Provide ONLY your answer and NOTHING else.
        CURRENT INFORMATION: {state["current_information"]}
        INITIAL QUESTION: {state["initial_question"]}
        ANSWER:
        """
        state['answering_agents'].append("Supervisor")
        state['prompts'].append(prompt)
        response = llm.invoke(prompt, use_cache=False)
        state['responses'].append(response)

        print(state['answering_agents'][-1])
        print(state['responses'][-1])
        print("------------------------------------------")
        print("\n\n")

        state["current_information"] = response.split("<|eot_id|>")[0].strip()

        RAW_KNOWLEDGE_BASE_ALL_INFO = [
            Document(page_content=state['all_information'])
        ]

        # Splitting documents into chunks
        docs_processed_all_info = utils.split_documents(
            256,  # We choose a chunk size adapted to our model
            RAW_KNOWLEDGE_BASE_ALL_INFO,
            tokenizer_name=EMBEDDING_MODEL_NAME,
        )

        docs_content_all_info = []
        for doc_processed in docs_processed_all_info:
            docs_content_all_info.append(doc_processed.page_content)

        # Encoding the documents
        vectors_all_info = encoder.encode(docs_content_all_info)
        df_database_all_info = pd.DataFrame(data=docs_content_all_info, columns=['text'])
        vector_dimension = vectors_all_info.shape[1]
        index_all_info = faiss.IndexFlatL2(vector_dimension)
        faiss.normalize_L2(vectors_all_info)
        index_all_info.add(vectors_all_info)
        state['all_information_index'] = index_all_info
        state['all_information_database'] = df_database_all_info
    
    if state["all_information"] != "":
        query = state["recent_response"]
        search_vector = encoder.encode(query)
        _vector = np.array([search_vector])
        faiss.normalize_L2(_vector)
        k = index_all_info.ntotal
        distances, ann = index_all_info.search(_vector, k=k)
        results = pd.DataFrame({'distances': distances[0], 'ann': ann[0]})
        merge = pd.merge(results, df_database_all_info, left_on='ann', right_index=True)
        relevant_all_info_text = ("\n\n").join(list(merge['text'].values[0:3]))
    else:
        relevant_all_info_text = ""


    # Check if state['Variables']['trialid'] exists
    if 'trialid' not in state['variables']:
        closest_trial_ids, closest_trial_id = get_most_relevant_clinical_trials(state["initial_question"])
        state['variables']['trialid'] = closest_trial_id
        state['variables']['trialid_list'] = closest_trial_ids

    prompt = f"""
    You are the main supervisor. Based on the INPUT, RECENT RESPONSE, CURRENT INFORMATION, ADDITIONAL INFOMRATION, determine the next step in the workflow. 

    
    INPUT: {state["initial_question"]}

    RECENT RESPONSE: {state["recent_response"]}

    SENDING AGENT: {state["sending_agent"]}

    CURRENT INFORMATION: {state["current_information"]}

    ADDITIONAL INFORMATION: {relevant_all_info_text}

    PREDEFINED NEXT AGENT: {state["predefined_next_agent"]}
    

    The possible next agents/steps are:
    - Trialist: Create or edit the target trial protocol when and if needed.
    - Informatician: Prepare and clean the data based on the target trial protocol and clinical literature. \
    The informatician needs a target trial and elibility criteria from the Trialist.
    - Clinician: Provide research or medical advice to answer the question from a medical professional. \
    The clinician can also provide advice of if trial emulation design or trial emulation results are reasonable and if more exploration is needed.
    - Statistician: Perform statistical analysis, machine learning, and or emulation to answer the question from computational experts. \
    This agent will perform processing, propensity score matching, estimating averate treatment effect, and Cox Proportional Hazards model to get results. \
    The statistician needs a cleaned dataset provided by the Informatician.

    As mentioned, based on the INPUT, RECENT RESPONSE, CURRENT INFORMATION, ADDITIONAL INFOMRATION, determine the next step in the workflow. 
    If PREDEFINED NEXT AGENT is provided, delegate the task to that agent but come up with a request based on the CURRENT INFORMATION and RECENT RESPONSE.
    If there is a previous response from the Clinican that includes results from trial emulation and a set of recommendations, \
    but the Informatician states that it is not possible to create a dataset that follows those recommendations, \
    the next step should "End".

    Only provide the NAME of the next agent to execute. For example, if the next agent is Clinician, ouptut "Clinician".
    In addition, provide a REQUEST for the next agent to execute. Output your answer as follows: "NEXT: NextAgent, REQUEST: Your request here". DO NOT PROVIDE ANY OTHER INFORMATION.
    The text of the request should be entirely self-contained and not require the reference of CURRENT INFORMATION or any other context variables.

    DO NOT PROVIDE ANY FORMATTING IN THE ANSWER.
    
    ANSWER:
    """

    if state['final_summary'] != "":
        if state['variables']['ec_optimization_results'] != "":
            state["final_summary"] = state["final_summary"] + "\n Eligibility Criteria Optimization Results:\n" + state['variables']['ec_optimization_results']
            state["next"] = "__end__"
            return state
        elif state['variables']['ec_patients_powerset_dataframe'] is None:
            state["next"] = "InformaticianAgentInvoker"
            state["predefined_next_agent"] = "Informatician"
            state["predefined_next_question"] = "Generate the required data for eligibility criteria optimization."
            state["ongoing_question"] = state["predefined_next_question"]
            state['answering_agents'].append("Supervisor")
            state['prompts'].append(prompt)
            state['responses'].append(state["ongoing_question"])

            print(state['answering_agents'][-1])
            print(state['responses'][-1])
            print("------------------------------------------")
            print("\n\n")

            # Clearing predefined_next_agent
            state["predefined_next_agent"] = ""
            state["predefined_next_question"] = ""
            # Clearing the recent response
            state["recent_response"] = ""
            return state
        else:
            pass

    if state["predefined_next_agent"] != "" and state['predefined_next_question'] != "":
        if state["predefined_next_agent"] == "Trialist":
            state["next"] = "TrialistInvoker"
        elif state["predefined_next_agent"] == "Informatician":
            state["next"] = "InformaticianAgentInvoker"
        elif state["predefined_next_agent"] == "Clinician":
            state["next"] = "ClinicianAgentInvoker"
        elif state["predefined_next_agent"] == "Statistician":
            state["next"] = "StatisticianAgentInvoker"
        state["ongoing_question"] = state["predefined_next_question"]
        state['answering_agents'].append("Supervisor")
        state['prompts'].append(prompt)
        state['responses'].append(state["ongoing_question"])
        
        print(state['answering_agents'][-1])
        print(state['responses'][-1])
        print("------------------------------------------")
        print("\n\n")

        # Clearing predefined_next_agent
        state["predefined_next_agent"] = ""
        state["predefined_next_question"] = ""
        # Clearing the recent response
        state["recent_response"] = ""
        return state

    elif state["predefined_next_agent"] == "Reviewer":
        state["next"] = "ClinicianAgentInvoker"
        state["ongoing_question"] = "Review the report and provide recommendations."
        state['answering_agents'].append("Supervisor")
        state['prompts'].append("Given the report, what is the next step?")
        state['responses'].append(state["ongoing_question"])
        
        print(state['answering_agents'][-1])
        print(state['responses'][-1])
        print("------------------------------------------")
        print("\n\n")

        # Clearing predefined_next_agent
        state["predefined_next_agent"] = ""
        state["predefined_next_question"] = ""
        # Clearing the recent response
        state["recent_response"] = ""
        return state

    else:
        state['answering_agents'].append("Supervisor")
        state['prompts'].append(prompt)
        response = llm.invoke(prompt, use_cache=False)
        state['responses'].append(response)
        
        print(state['answering_agents'][-1])
        print(state['responses'][-1])
        print("------------------------------------------")
        print("\n\n")

    try:
        next_agent_response = response.split("NEXT:")[1].split("REQUEST:")[0].strip()
        request = response.split("REQUEST:")[1].strip()
    except:
        next_agent_response = "End"
        request = ""
    
    # Setting question for next agent, this ongoing_question is only set by the MainSupervisor
    # Preserve the clinician response
    if state['sending_agent'] == 'Clinician':
        state["ongoing_question"] = "Clinician Answer: " + state["recent_response"] + " Request: " + request
    else:
        state["ongoing_question"] = request
    
    # Clearing predefined_next_agent
    state["predefined_next_agent"] = ""
    state["predefined_next_question"] = ""

    # Clearing the recent response
    state["recent_response"] = ""
    
    if "Clinician" in next_agent_response:
        state["next"] = "ClinicianAgentInvoker"
    elif "Statistician" in next_agent_response:
        state["next"] = "StatisticianAgentInvoker"
    elif "Trialist" in next_agent_response:
        state["next"] = "TrialistInvoker"
    elif "Informatician" in next_agent_response:
        state["next"] = "InformaticianAgentInvoker"
    elif "End" in next_agent_response:
        if state["final_summary"] == "":
            if state["current_report"] != "":
                state["final_summary"] = state["current_report"]
        state["next"] = "__end__"
    else:
        state["next"] = "__end__"

    return state



def Trialist(state: TrialistState):
    
    state['sending_agent'] = "Trialist"
    
    if 'trialid' not in state['variables']:
        state['recent_response'] = f"Trial ID is not specified."
        state['sending_agent'] = "Trialist"
        state['next'] = "__end__"
        return state 

    trial_pkl_path = osp.join('trials', f"{state['variables']['trialid']}", f"trial_info_{state['variables']['trialid']}.pkl")
    if not osp.exists(trial_pkl_path):
        parse_clinical_trial(state['variables']['trialid'])
    
    # load trial info
    with open(trial_pkl_path, "rb") as fin:
        trial_info = pickle.load(fin)
    
    state['variables']['inclusion_criteria'] = [item.text for item in trial_info.inclusion_criteria]
    state['variables']['exclusion_criteria'] = [item.text for item in trial_info.exclusion_criteria]
    if len(trial_info.treatments) == 1:
        state['variables']['treatment_definition'] = [item.text for item in trial_info.treatments]
    elif len(trial_info.treatments) == 2:
        state['variables']['treatment_definition'] = [f"Treated: {trial_info.treatments[0].text}; Control: {trial_info.treatments[1].text}"]
    else:
        assert False, "Trialist should only have two treatments, one for treated and one for control."
    state['variables']['outcome_definition'] = [item.text for item in trial_info.outcomes]

    state["recent_response"] = f"""
        Parsed trial information is as follows and no need to go back to Trialist again even Clinician recommended.
        Inclusion Criteria: {state['variables']['inclusion_criteria']};
        Exclusion Criteria: {state['variables']['exclusion_criteria']};
        Treatment Definition: {state['variables']['treatment_definition']};
        Outcome Definition: {state['variables']['outcome_definition']};
    """
    state['prompts'].append("Come up with the information about the trial to emulate.")
    state['responses'].append(state["recent_response"])
    state['answering_agents'].append("Trialist")
    
    print(state['answering_agents'][-1])
    print(state['responses'][-1])
    print("------------------------------------------")
    print("\n\n")
    
    state["next"] = "__end__"

    return state



def Clinician(state: ClinicianState):
    """Medical professional agent."""
    """Can look at pubmed abstracts and own expertise to answer questions"""

    # Getting relevant text from pdf literature
    if 'compiled_relevant_texts' not in state['variables']:
        state['variables']['compiled_relevant_texts'] = pd.DataFrame(columns=['text'])
    combined_texts, state = get_relevant_info_from_lit_and_pubmed(state)

    if 'number_time_provided_recs' not in state['variables']:
        state['variables']['number_time_provided_recs'] = 0

    if state['ongoing_question'] == "Review the report and provide recommendations.":
        if state['variables']['number_time_provided_recs'] >= 2:
            response = "Clinician has already provided recommendations two times. No further recommendations will be provided. CREATE FINAL REPORT"
        else:
            review_prompt = f"""
            You are a medical professional with expertise in a wide variety of medical fields. You are tasked with analyzing the given reports and providing recommendations \
            Based on the REPORT and your own expertise, you need to DECIDE whether to provide recommendations or a final report. \

            REPORT: {state['current_report']}

            ---------------------

            Follow the tips:
            - If you believe the results are satisfactory, and the discrepancies are minor or cannot be solved, opt to NOT give recommendations.
            - Your theoretical recommendations should be able to be carried out within the MIMIC-IV database and no other external data. If they cannot, then do NOT choose to give recommendations.
            - If the theoretical recommendations are too broad or create too much change, just opt to include them as Limitations in the final report. \
            - Prefer creating a final report if there are no urgent changes needed. \

            The Expected Structure Output Should Be Either:
            "CREATE FINAL REPORT"
            or
            "PROVIDE RECOMMENDATIONS"

            DO NOT PROVIDE ANY FORMATTING IN THE ANSWER.
            """

            state['answering_agents'].append("Clinician")
            state['prompts'].append(review_prompt)
            response = llm.invoke(review_prompt, use_cache=False)
            state['responses'].append(response)

            print(state['answering_agents'][-1])
            print(state['responses'][-1])
            print("------------------------------------------")
            print("\n\n")

        if "CREATE FINAL REPORT" in response:
            final_review_prompt = f"""
                You are a clinician with medical expertise. Create a final report based on the CURRENT REPORT. \
                Provide a comprehensive summary of the trial emulation process, results, and recommendations for future research. \
                
                You are given the following: 

                CURRENT REPORT: {state['current_report']}
                
                You need output a complete report. It should include a clear and informative title page that outlines the study’s title, \
                followed by an abstract summarizing the objective, methods, key results, and conclusions. The introduction should provide background \
                on the original trial, the rationale for emulation, the study objectives, and key challenges. A detailed description of the data sources is essential, \
                including the dataset characteristics, inclusion/exclusion criteria, and data quality assessments. The study design should outline how the emulation \
                mirrors the original trial, specifying eligibility criteria, treatment strategies, outcomes, and follow-up \
                periods. The statistical methods section should describe the causal inference approaches, confounder adjustment techniques, \
                outcome modeling, and sensitivity analyses. The results section should present baseline characteristics, treatment effect estimates, \
                comparisons with the original trial, and findings from sensitivity analyses, supported by tables and visualizations. In the discussion, \
                key findings should be interpreted in the context of consistency with the original trial, alongside the study’s strengths, limitations, \
                and implications for future research. The conclusion should summarize the study’s impact and contributions.

                Expected Structure Output:
                "FINAL REPORT: <Your final report here>"

                Provide ONLY your answer and NOTHING else.
                """
        
            state['answering_agents'].append("Clinician")
            state['prompts'].append(final_review_prompt)
            response = llm.invoke(final_review_prompt, use_cache=False)
            state['responses'].append(response)

            print(state['answering_agents'][-1])
            print(state['responses'][-1])
            print("------------------------------------------")
            print("\n\n")

            if "FINAL REPORT" in response:
                response = response.split("FINAL REPORT")[1].strip()
                state['final_summary'] = response
                state["recent_response"] = response.strip()
                state['sending_agent'] = "Clinician"
                state["next"] = "__end__"
                return state


        if "PROVIDE RECOMMENDATIONS" in response:
            state['variables']['number_time_provided_recs'] += 1
            recommendations_prompt = f"""
            You are a clinician with medical expertise. Based on the CURRENT REPORT and CURRENT INFORMATION, provide recommendations for further analysis or changes to the trial emulation process. \
            Your recommendations should be specific, actionable, and relevant to the trial emulation process. \

            CURRENT REPORT: {state['current_report']}


            CURRENT INFORMATION: {state['current_information']}


            Here is an expected **Example Output**:
            '''
            RECOMMENDATIONS:
            <Treatment_Effect>Original Hazard Ratio: 0.82 (95% CI: 0.70 - 0.95)</Treatment_Effect>   
            <Comparison>Results are consistent within the confidence intervals, suggesting alignment.</Comparison>   
            <Potential_Discrepancy>Minor difference in point estimates</Potential_Discrepancy>   
            <Reason>Differences may stem from variations in patient populations or eligibility criteria in our trial emulation.</Reason>   
            <Recommendation>No major concerns. Proceed with the current analysis, but include a note discussing potential sources of discrepancy.</Recommendation> 
            Reasons of discrepancies: 
            1. Eligibility Criteria Mismatch: 
            Our trial emulation's EC is less restrictive compared to the Original Trial. 
            <Solution>Add additional eligibility criteria to better align with the Original Trial, such as stricter temporal conditions or exclusion criteria.</Solution> 
            2. Causal Estimation Method Inaccuracy: 
            The current causal inference method might not fully account for confounding.   
            <Solution>Consider switching causal estimation methods, such as propensity score matching and G-computation.</Solution> 
            3. The smaller sample size in the emulated trial might lead to unstable estimates.   
            <Solution>Relax certain eligibility criteria (e.g., extend temporal windows) to increase sample size while monitoring covariate balance.</Solution> 
            PREDEFINED AGENT: Informatician
            '''

            If there are recommendations, you may want to send the recommendations to **ONE** specific agent. \
            If so, provide the agent's name at the end of the output (in addition to the answer) as PREDEFINED AGENT: AgentName \
            Here are the possible agents: \
            - Trialist: Create or edit the target trial protocol when and if needed. \
            - Informatician: Prepare and clean the data based on the target trial protocol, clinical literature. The clinician deals with curating a new dataset based on recommendations. \
            - Statistician: Perform statistical analysis, machine learning, and or emulation to answer the question from computational experts. \

            Follow the tips:
            - If your recommendations require the creation of a new dataset, the PREDEFINED AGENT should be the Informatician.
            -- Examples of creating a new dataset:
                --- Incorporate subgroup analyses focusing on patient stratifications such as illness severity 
                --- Relax Temporal Eligibility Criteria for Trial Emulation
            - If you believe the results are satisfactory, and the discrepancies are minor or cannot be solved, make a complete report including the motivation, background, results, and discussion of discrepancies.
            - Your recommendations should be able to be carried out within the MIMIC-IV database and no other external data. \
            - If the recommendations are too broad or create too much change, just include them as Limitations in the final report. \
            - Prefer putting reccomendations in the Limitations. \

            Provide ONLY your answer and NOTHING else.
            """

            state['answering_agents'].append("Clinician")
            state['prompts'].append(recommendations_prompt)
            response = llm.invoke(recommendations_prompt, use_cache=False)
            state['responses'].append(response)

            print(state['answering_agents'][-1])
            print(state['responses'][-1])
            print("------------------------------------------")
            print("\n\n")

            if "PREDEFINED AGENT" in response:
                predefined_next_agent = response.split("PREDEFINED AGENT: ")[1].split("\n")[0].strip()
                response = response.split("PREDEFINED AGENT: ")[0]

                # Ask llm to provide a structured list of steps for the predefined agent to take
                prompt = f"""
                Provide a structured list of steps for the {predefined_next_agent} to take based on the given response.

                The response is as follows:
                {response}

                Please provide a clear and concise list of steps for the {predefined_next_agent} to follow in response to the given information.

                Structure your response as follows:
                1. Step 1
                2. Step 2
                3. Step 3
                ...

                Provide ONLY your answer and NOTHING else.
                """

                state['answering_agents'].append("Clinician")
                state['prompts'].append(prompt)
                steps_response = llm.invoke(prompt, use_cache=False)
                state['responses'].append(response)

                print(state['answering_agents'][-1])
                print(state['responses'][-1])
                print("------------------------------------------")
                print("\n\n")

                state['predefined_next_agent'] = predefined_next_agent
                state['predefined_next_question'] = steps_response
                
                state["recent_response"] = steps_response.strip()
                state['sending_agent'] = "Clinician"
                state["next"] = "__end__"

        
        state["recent_response"] = response.strip()
        state['sending_agent'] = "Clinician"
        state["next"] = "__end__"

        return state


    prompt = f"""
    You are a medical professional with expertise in a wide variety of medical fields. \
    Based on the QUESTION and MEDICAL LITERATURE, and your own expertise provide an answer to the question. \
    You might get requests (INPUT) about doing research, providing guidance of how to change trial design, or providing medical advice.
    You may want to send the answer to a specific agent. It will likely be the SENDING AGENT. \
    If so, provide the agent's name at the end of the output (in addition to the answer) as PREDEFINED AGENT: AgentName \
    Here are the possible agents: \
    - Statistician: Perform statistical analysis, machine learning, and or emulation to answer the question from computational experts. \
    - Trialist: Create or edit the target trial protocol when and if needed. \
    - Informatician: Prepare and clean the data based on the target trial protocol and clinical literature. \
   
    
    You might get other questions as well that are not covered in the examples above. Provide the answers in a structured format. \
    The output will need to be parsed by the next agent to extract the answer. \
    The output for the provided examples should be structured as shown in the above examples.
    ONLY provide the answer to the question. DO NOT provide any other information and be concise.

    DO NOT PROVIDE ANY ADDITIONAL FORMATTING IN THE ANSWER.

    SENDING AGENT: {state["sending_agent"]}

    MEDICAL LITERATURE: {combined_texts}

    QUESTION: {state["ongoing_question"]}


    ANSWER:
    """

    state['answering_agents'].append("Clinician")
    state['prompts'].append(prompt)
    response = llm.invoke(prompt, use_cache=False)
    state['responses'].append(response)

    print(state['answering_agents'][-1])
    print(state['responses'][-1])
    print("------------------------------------------")
    print("\n\n")

    if "PREDEFINED AGENT: " in response:
        state["predefined_next_agent"] = response.split("PREDEFINED AGENT: ")[1].split("\n")[0].strip()
        if "ANSWER:" in response:
            response = response.split("ANSWER: ")[1].split("PREDEFINED AGENT: ")[0]
        response = response.split("PREDEFINED AGENT: ")[0]
        state["predefined_next_question"] = response

    if "INCLUDED COVARIATES" in response:  # jump back to Informatician ### QUESTION
        state['predefined_next_question'] = response
        state['predefined_next_agent'] == "Informatician"

    if ("STRATIFY ON" in response) and ("LEVEL" in response):
        # Replace "**STRATIFY ON**" with "STRATIFY ON"
        response = response.replace("**STRATIFY ON**", "STRATIFY ON")
        response = response.replace("**LEVEL**", "LEVEL")
        response = "SUBGROUP ANALYSIS\n\n" + response
        state['predefined_next_question'] = response
        state['predefined_next_agent'] = "Statistician"

    if "UNBALANCED COVARIATES" in response:
        response = response.replace("**UNBALANCED COVARIATES**", "UNBALANCED COVARIATES")
        state['predefined_next_question'] = response
        state['predefined_next_agent'] = "Statistician"
    
    if "ANSWER:" in response:
        response = response.split("ANSWER: ")[1].strip()

    state["recent_response"] = response.strip()

    if "FINAL SUMMARY" in state["recent_response"]:
        state["final_summary"] = state["recent_response"].split("FINAL SUMMARY:")[1].strip()
    
    state['sending_agent'] = "Clinician"
    state["next"] = "__end__"
    return state



def Informatician(state: InformaticianState):
    """
    Informatician Agent to process raw CSV data, compute required variables,
    and prepare a cleaned DataFrame for trial emulation.
    """

    state['sending_agent'] = "Informatician"

    # 1. check the availability of required variables
    if 'inclusion_criteria' not in state['variables'] or state['variables']['inclusion_criteria'] is None:
        state['recent_response'] = "inclusion_criteria is not available."
        state['sending_agent'] = "Informatician"
        state['next'] = "__end__"
        return state
    if 'exclusion_criteria' not in state['variables'] or state['variables']['exclusion_criteria'] is None:
        state['recent_response'] = "exclusion_criteria is not available."
        state['sending_agent'] = "Informatician"
        state['next'] = "__end__"
        return state
    if 'treatment_definition' not in state['variables'] or state['variables']['treatment_definition'] is None:
        state['recent_response'] = "treatment_definition is not available."
        state['sending_agent'] = "Informatician"
        state['next'] = "__end__"
        return state
    if 'outcome_definition' not in state['variables'] or state['variables']['outcome_definition'] is None:
        state['recent_response'] = "outcome_definition is not available."
        state['sending_agent'] = "Informatician"
        state['next'] = "__end__"
        return state    
    

    # 2. load df and cols information 
    trial_info = {
        'treatment_definition': [clean_ec(_) for _ in state['variables']['treatment_definition']],
        'outcome_definition': [clean_ec(_) for _ in state['variables']['outcome_definition']],
        'inclusion_criteria': [clean_ec(_) for _ in state['variables']['inclusion_criteria']],
        'exclusion_criteria': [clean_ec(_) for _ in state['variables']['exclusion_criteria']]
    }
    informatician = InformaticianTool(trialID=state['variables']['trialid'], trial_info=trial_info, llm=llm, dataset=state['variables'].get('dataset', None), root_dir='./trials')
    df, cols_dict = informatician.get_df_dict()
    # df_notes = informatician.get_notes()
    
    state['variables']['cleaned_dataframe'] = df
    state['variables']['cols_dict'] = cols_dict
    # state['variables']['Covariates'] = cols_dict['covariate_cols']
    state['variables']['Treatment'] = cols_dict['treatment_cols']
    state['variables']['Outcome'] = cols_dict['outcome_cols']
    state['variables']['Duration']  = cols_dict['duration_cols']
    state['variables']['patient_id']  = cols_dict['patient_id']

    # for handling immortal_time_bias  
    state['variables']['treatment_time_cols']  = cols_dict['treatment_time_cols'] if 'treatment_time_cols' in cols_dict else None

    # Defining state['variables']['ec_patients_powerset_dataframe'] for eligibility criteria optimization for the future
    state['variables']['ec_patients_powerset_dataframe'] = None

    # 3. prepare the eligibility criteria optimization
    if 'ELIGIBILITY CRITERIA OPTIMIZATION' in state["ongoing_question"].upper():

        dict_ec = informatician.get_permuted_ec_for_optimize()
        state['variables']['patient_id'] = cols_dict['patient_id']
        state['variables']['big_dataframe'] = dict_ec['variables']['big_dataframe']
        state['variables']['ec_patients_powerset_dataframe'] = dict_ec['variables']['ec_patients_powerset_dataframe']

        state['sending_agent'] = "Informatician"
        state["predefined_next_question"] = "Hi Statistician, you need to conduct eligibility criteria optimization."
        state["predefined_next_agent"] = "Statistician"
        state["next"] = "__end__"
        return state

    # 4. process included covariates
    if 'included_covariates' not in state['variables'] or state['variables']['included_covariates'] is None: # ask Clinician to set covariates that will be built for dataframe
        
        if ("flag_asked_clinician_include_covariates" in state["variables"] and state["variables"]["flag_asked_clinician_include_covariates"]) or 'INCLUDED COVARIATES' in state['predefined_next_question'] and state['sending_agent'] == "Clinician": # process response from Clinician
            covariates_to_include = []
            try:
                covariates_to_include = state['recent_response'].split("INCLUDED COVARIATES:")[1].strip().split(",")
            except:
                pass
            covariates_to_include = [_.strip() for _ in covariates_to_include]
            state['variables']['included_covariates'] = [col for col in cols_dict['covariate_cols'] if col in covariates_to_include] # select the covariates that are in the list
            state["predefined_next_question"] = ""

        else:  # ask Clinician
            state["variables"]["flag_asked_clinician_include_covariates"] = True
            state['sending_agent'] = "Informatician"
            state["predefined_next_question"] = f"""
                Hi Clinician, you need to answer which covariates should be used for build the dataframe for analysis. 
                
                You need to select a subset or the entire set (as MORE as possible) from the available covariate list that can serve as the covariates for samples in the dataset used for conducting trial emulation. After you

                The available covariates are: {cols_dict['covariate_cols']}

                In your answer, you need to include two lines. In the first line, you need to provide a list of the covariates to be selected in the following format:
                INCLUDED COVARIATES: <Covariate1>,<Covariate2>,<Covariate3> ... where <Covariate1> is the covariate name. And then in the second line, uou also need to specify the PREDEFINED AGENT to be Informatician, namely output "PREDEFINED AGENT: Informatician"
                Remember that you do not output any other context in the output except these two lines.

                Do not change the covariate name in your answer and only use the covariate name that is shown in the available covariates that I give to you.
                
                Do **not** include additional information or context in the response. Do not provide an explanations or reasoning for the selection of covariates. ONLY provide the Output in Expected Structure Output.

                Expected Structure Output:
                INCLUDED COVARIATES: <Covariate1>|<Covariate2>|<Covariate3>|<Covariate4>|<Covariate5>
                PREDEFINED AGENT: Informatician
            """
            state['predefined_next_agent'] = "Clinician"
            state["next"] = "__end__"
            return state

    state['variables']['Covariates'] = state['variables']['included_covariates']

    state["recent_response"] = f"""
        The built dataframe is ready for analyzing in Statistician. 
    """
    state['predefined_next_agent'] = "Statistician"
    state['predefined_next_question'] = "Conduct statistical analysis using the data provided."

    # Quality check
    if state['variables']['Covariates'] is None or len(state['variables']['Covariates']) <= 3:
        state['variables']['Covariates'] = cols_dict['covariate_cols']
        state["recent_response"] += f"""The selection or adjustment on covariates failed or very few covariates are recommended, so Informatician includes all covariates, so that do not go back to Informatician again even being recommended by the others."""

    # loop check
    if 'cnt_informatician_response' not in state['variables'] or state['variables']['cnt_informatician_response'] is None:
        state['variables']['cnt_informatician_response'] = 0
    else:
        state['variables']['cnt_informatician_response'] += 1
    if state['variables']['cnt_informatician_response'] > 10:  # TODO: might change in the future
        state["recent_response"] += f"""The Informatician has reach the maximum number of response so that do not go back to Informatician again even being recommended by the others."""

    # save responses
    state['prompts'].append("Create the dataframe for the trial emulation.")
    state['responses'].append(state["recent_response"])
    state['answering_agents'].append("Statistician")

    print(state['answering_agents'][-1])
    print(state['responses'][-1])
    print("------------------------------------------")
    print("\n\n")

    state["next"] = "__end__"
    return state



def Statistician(state: StatisticianState):
    """
    Enhanced Statistician Agent to build trial emulation protocol, select the best covariate balancing method,
    conduct survival analysis using various models, and generate relevant result plots.
    """

    list_of_outcomes = state['variables']['Outcome']
    list_of_durations = state['variables']['Duration']
    list_of_outcomes = [outcome.lower() for outcome in list_of_outcomes]
    list_of_durations = [duration.lower() for duration in list_of_durations]
    
    outcome_definitions = state['variables']['outcome_definition'] + [_.replace('adverseevent_', '') for _ in state['variables']['Outcome'] if 'adverseevent' in _]
    state['variables']['outcome_definitions'] = outcome_definitions

    if 'completed_outcomes' not in state['variables']:
        state['variables']['completed_outcomes'] = []


    # ELIGIBILITY CRITERIA OPTIMIZATION
    if 'ec_optimization_results' not in state['variables']:
        state['variables']['ec_optimization_results'] = ""

    if "ELIGIBILITY CRITERIA OPTIMIZATION" in state['ongoing_question'].upper():
        shapley_value, name_rules, hrs_for_all_combos = perform_ec_optmization(state)
        ec_optimixation_str = ""
        for i in range(len(name_rules)):
            if shapley_value[i] > 0:
                ec_optimixation_str += f"{name_rules[i]}: is associated with an increase in HR, with a SHAP value of {shapley_value[i]}\n"
            elif shapley_value[i] < 0:
                ec_optimixation_str += f"{name_rules[i]}: is associated with a decrease in HR, with a SHAP value of {shapley_value[i]}\n"
            else:
                ec_optimixation_str += f"{name_rules[i]}: has no association with HR, with a SHAP value of {shapley_value[i]}\n"
        state['variables']['ec_optimization_results'] = ec_optimixation_str
        state['variables']['ec_shapley_value'] = shapley_value
        state['variables']['ec_name_rules'] = name_rules
        state['variables']['flag_ec_optimization_performed'] = True
        state['next'] = "__end__"
        return state

    
    # Initiating loop for initial analysis for each outcome
    if 'current_outcome_index' not in state['variables']:
        # If current_outcome_index is not in state, initialize it to 0
        state['variables']['current_outcome_index'] = 0

    while state['variables']['current_outcome_index'] < len(list_of_outcomes):

        list_of_outcome = list_of_outcomes[state['variables']['current_outcome_index']]
        list_of_duration = list_of_durations[state['variables']['current_outcome_index']]
        outcome_definition = outcome_definitions[state['variables']['current_outcome_index']]

        if isinstance(list_of_outcome, str):
            list_of_outcome = [list_of_outcome]
        if isinstance(list_of_duration, str):
            list_of_duration = [list_of_duration]
        if isinstance(outcome_definition, str):
            outcome_definition = [outcome_definition]
        
        if list_of_outcome[0] not in state['variables']:
            state['variables'][list_of_outcome[0]] = {}

        
        # SUBGROUP ANALYSIS
        if "SUBGROUP ANALYSIS" in state['ongoing_question']:
            state = _handle_subgroup_analysis(state, list_of_outcome, list_of_duration)
            if state['next'] == "__end__":
                return state
            else:
                continue

        # NON SUBGROUP ANALYSIS
        else:
            # Load the cleaned DataFrame
            if state['variables']['cleaned_dataframe'] is None:
                state['recent_response'] = "Cleaned DataFrame is not available."
                state['sending_agent'] = "Statistician"
                state['next'] = "__end__"
                return state

            dataframe = state['variables']['cleaned_dataframe']

            # make all columns lower case
            dataframe.columns = [col.lower() for col in dataframe.columns]

            if 'ignore_covariates' not in state['variables'][list_of_outcome[0]]:
                state['variables'][list_of_outcome[0]]['ignore_covariates'] = []
            if 'add_covariates' not in state['variables'][list_of_outcome[0]]:
                state['variables'][list_of_outcome[0]]['add_covariates'] = []
            if 'drop_covariates' not in state['variables'][list_of_outcome[0]]:
                state['variables'][list_of_outcome[0]]['drop_covariates'] = []

            if 'number_times_request_drop_covariates' not in state['variables'][list_of_outcome[0]]:
                state['variables'][list_of_outcome[0]]['number_times_request_drop_covariates'] = 0

            # Based on the most recent response, determine if there are any covariates to drop
            state, covariates_to_add, covariates_to_drop, covariates_to_ignore = _parse_covariate_modifications(state,
                                                                                                                dataframe,
                                                                                                                list_of_outcome,
                                                                                                                llm)

            ## Informatician needs to store the relevant column names as a list to state variables
            list_of_covariates = state['variables']['Covariates']
            list_of_treatment = state['variables']['Treatment']

            # if list_of_treatment is a string, convert it to a list
            if isinstance(list_of_treatment, str):
                list_of_treatment = [list_of_treatment]

            # make all elements in list_of_covariates lower case
            list_of_covariates = [covariate.lower() for covariate in list_of_covariates]
            # make all elements in list_of_treatment lower case
            list_of_treatment = [treatment.lower() for treatment in list_of_treatment]
            # make all elements in list_of_outcome lower case
            list_of_outcome = [outcome.lower() for outcome in list_of_outcome]
            # make all elements in list_of_duration lower case
            list_of_duration = [duration.lower() for duration in list_of_duration]
            
            # Might need to change code in the future
            # Drop the covariates if needed in list_of_covariates
            new_list_of_covariates = []
            for covariate in list_of_covariates:
                if covariate.lower() not in covariates_to_drop:
                    new_list_of_covariates.append(covariate)
            list_of_covariates = new_list_of_covariates

            # Add the covariates if needed in list_of_covariates
            # Check if covariates_to_add is in the dataframe
            final_covariates_to_add = []
            for covariate in covariates_to_add:
                if covariate in dataframe.columns:
                    final_covariates_to_add.append(covariate)
            list_of_covariates = list(set(list_of_covariates + final_covariates_to_add))

            identifier_columns = ['subject_id', 'stay_id']

            df, categorical_covariates, continuous_covariates, list_of_treatment, list_of_outcome, list_of_duration = clean_data(list_of_covariates,
                                                                                                                                list_of_treatment,
                                                                                                                                    list_of_outcome,
                                                                                                                                    list_of_duration,
                                                                                                                                    identifier_columns,
                                                                                                                                        dataframe)
            

            # if there are any remaining columns with all NaN values, drop them
            columns_with_nan = df.columns[df.isna().all()].tolist()
            df.drop(columns=columns_with_nan, inplace=True)
            # get overlap between columns_with_nan and list_of_covariates
            columns_with_nan = [col for col in columns_with_nan if col in list_of_covariates]
            state['variables'][list_of_outcome[0]]['drop_covariates'].extend(columns_with_nan)
            state['variables'][list_of_outcome[0]]['drop_covariates'] = list(set(state['variables'][list_of_outcome[0]]['drop_covariates']))
            covariates_to_drop = state['variables'][list_of_outcome[0]]['drop_covariates']
            # Drop the covariates if needed in list_of_covariates
            list_of_covariates = [covariate for covariate in list_of_covariates if covariate.lower() not in covariates_to_drop]
            categorical_covariates = [covariate for covariate in categorical_covariates if covariate.lower() not in covariates_to_drop]
            continuous_covariates = [covariate for covariate in continuous_covariates if covariate.lower() not in covariates_to_drop]


            # Step 1: Build Trial Emulation Protocol
            trial_protocol = build_trial_emulation_protocol(state)

            break_flag = False
            results = {}
            plots = {}

            # Step 2.5 Perform Cloning If Possible to Fix Immortal Time Bias
            id_col = 'stay_id' # need to be changed in the future
            df = adjust_immortal_time_bias(df, id_col, treat_time_col=list_of_treatment[0], outcome_col=list_of_outcome[0], followup_end_col=list_of_duration[0], covariates=categorical_covariates + continuous_covariates, stabilize=True)

            # Step 2: Select Covariate Balancing Method
            balancing_method, balancing_rationale, survival_methods, survival_rationale = select_methods(state, df, trial_protocol, list_of_treatment, list_of_outcome, list_of_duration)
            state['variables'][list_of_outcome[0]]['balancing_method'] = balancing_method
            state['variables'][list_of_outcome[0]]['balancing_rationale'] = balancing_rationale
            state['variables'][list_of_outcome[0]]['survival_methods'] = survival_methods
            state['variables'][list_of_outcome[0]]['survival_rationale'] = survival_rationale

            # Step 3: Conduct Covariate Balancing
            balanced_df, balancing_str_info, break_flag, results, scaler = apply_covariate_balancing(state, balancing_method, df, categorical_covariates, continuous_covariates, state['variables'][list_of_outcome[0]]['ignore_covariates'], list_of_treatment, list_of_outcome, outcome_definition, break_flag, results)
            if scaler is not None:
                # Inverse transform the balanced dataframe
                balanced_df[continuous_covariates] = scaler.inverse_transform(balanced_df[continuous_covariates])

            if break_flag and state['variables'][list_of_outcome[0]]['number_times_request_drop_covariates'] <= 5: # If the covariates are not balanced, stop the analysis
                balances = results['Balances after performing balancing']
                unbalanced_covariates = list(balances[balances > 0.1].index)
                unbalanced_covariates_str = ""
                for covariate in unbalanced_covariates:
                    unbalanced_covariates_str += f"<Unbalanced_Covariate>{covariate}</Unbalanced_Covariate>\n"
                issue = f'''
                Hi Clinician, I noticed the ratio of unbalanced covariates is quite high, which may impact the validity of our estimation results.  
                I’d like to ask for your input: Can we drop these unbalanced covariates if they are less clinically significant? 
                Alternatively, can you recommend substitute covariates that might capture similar clinical signals while improving balance?  
                Here are the unbalanced covariates I identified: 
                {unbalanced_covariates_str}

                Expected Structure Output:
                UNBALANCED COVARIATES
                <Unbalanced_Covariate>|<DROP or SUBSTITUTE>|<Substitute Covariate or N/A>|<Justification>
                '''
                # store in state
                state['recent_response'] = issue
                state['sending_agent'] = "Statistician"
                state['predefined_next_agent'] = "Clinician"
                state['predefined_next_question'] = issue
                state['next'] = "__end__"
                state['variables'][list_of_outcome[0]]['number_times_request_drop_covariates'] += 1
                return state

            # Step 4: Conduct Survival Analysis
            results, plots, survival_str_info = conduct_survival_analysis(state, survival_methods, df, balanced_df, categorical_covariates, continuous_covariates, list_of_treatment, list_of_outcome, list_of_duration, outcome_definition, results, plots)

            # results_str = f"{balancing_str_info}\n{survival_str_info}"
            results_str = f"{survival_str_info}"

            # Making results for the specific outcome
            state['variables'][list_of_outcome[0]]['trial_protocol'] = trial_protocol
            state['variables'][list_of_outcome[0]]['balancing_information'] = balancing_str_info
            state['variables'][list_of_outcome[0]]['balanced_df'] = balanced_df
            state['variables'][list_of_outcome[0]]['subgroup_analysis_dict'] = {}
            state['variables'][list_of_outcome[0]]['subgroup_analysis_dict']['No Stratification'] = results_str
            state['variables'][list_of_outcome[0]]['categorical_covariates'] = categorical_covariates
            state['variables'][list_of_outcome[0]]['continuous_covariates'] = continuous_covariates
            state['variables'][list_of_outcome[0]]['results'] = results

            # Check if Subgroup Analysis is needed
            current_pval = None
            if 'Cox Proportional Hazards Summary' in results:
                current_pval = results['Cox Proportional Hazards Summary']['p'][list_of_treatment[0]]

            if (current_pval is not None) and (current_pval >= 0.05):
                # Results are not significant so we want to ask the clinican to ask for a stratification on a covariate
                # Provide all the continuous covariates and their ranges
                # Provide all the categorical covariates and their unique values
                continuous_covariates_records = {}
                for col in continuous_covariates:
                    continuous_covariates_records[col] = [balanced_df[col].min(), balanced_df[col].max()]
                categorical_covariates_records = {}
                for col in categorical_covariates:
                    categorical_covariates_records[col] = balanced_df[col].unique().tolist()
                issue = f'''
                Clinician, the current hazard ratio is not significant. Please provide a covariate to stratify on. Also provide the level to stratify on.

                You are given the following covariates and their ranges:

                Continuous Covariates:
                {continuous_covariates_records}

                Categorical Covariates:
                {categorical_covariates_records}
                
                Expected Structure Output:
                STRATIFY ON: <Covariate>
                LEVEL: <Level>

                The Covariate should be one of the covariates provided above. It should be spelled the EXACT same way as provided above.
                Level should be a value within the range of the covariate. \
                Level **MUST** be a number. Do not output a placeholder or a string. It will be parsed and used for stratification.

                Do not provide additional information or context in the response. No explanation is needed. Provide BOTH "STRATIFY ON" and "LEVEL" in the response.
                ONLY provide **ONE** stratification.
                '''
                state['recent_response'] = issue
                state['sending_agent'] = "Statistician"
                state['predefined_next_agent'] = "Clinician"
                state['predefined_next_question'] = issue
                state['next'] = "__end__"
                return state
            else:
                # If the results are significant, we can proceed to the next outcome
                state['variables']['completed_outcomes'].append(list_of_outcome[0])
                state['variables']['current_outcome_index'] += 1
                state['recent_response'] = f"Analysis for {list_of_outcome[0]} is completed. Proceeding to the next outcome."
                state['ongoing_question'] = f"Proceeding to the next outcome analysis."
                state['next'] = "Statistician"


    # Step 5: Write the full summary of results and context
    state = _generate_final_report(state, llm)

    if 'cnt_statistician_response' not in state['variables'] or state['variables']['cnt_statistician_response'] is None:
        state['variables']['cnt_statistician_response'] = 0
    else:
        state['variables']['cnt_statistician_response'] += 1
    
    if state['variables']['cnt_statistician_response'] > 10:  # TODO: might change in the future
        state["recent_response"] += f"""The Statistician has reached the maximum number of response so that do not go back to Statistician again even being recommended by the others."""

    return state

In [ ]:
# Build the medical graph
medical_builder = StateGraph(ClinicianState)
medical_builder.add_node(Clinician)
medical_builder.add_edge(START, "Clinician")
medical_builder.add_conditional_edges("Clinician", lambda state: state["next"])
medical_graph = medical_builder.compile()
display(Image(medical_graph.get_graph().draw_mermaid_png()))

# Build the statistician graph
statistician_builder = StateGraph(StatisticianState)
statistician_builder.add_node(Statistician)
statistician_builder.add_edge(START, "Statistician")
statistician_builder.add_conditional_edges("Statistician", lambda state: state["next"])
statistician_graph = statistician_builder.compile()
display(Image(statistician_graph.get_graph().draw_mermaid_png()))

# Build the informatician graph (assuming it's a single function invoker)
informatician_builder = StateGraph(InformaticianState)
informatician_builder.add_node(Informatician)
informatician_builder.add_edge(START, "Informatician")
informatician_builder.add_conditional_edges("Informatician", lambda state: state["next"])
informatician_graph = informatician_builder.compile()
display(Image(informatician_graph.get_graph().draw_mermaid_png()))

# Build the trial expert graph
trial_builder = StateGraph(TrialistState)
trial_builder.add_node(Trialist)
trial_builder.add_edge(START, "Trialist")
trial_builder.add_conditional_edges("Trialist", lambda state: state["next"])
trial_graph = trial_builder.compile()
display(Image(trial_graph.get_graph().draw_mermaid_png()))


# Build node to invoke medical graph
def ClinicianAgentInvoker(state: TopAgentState):
    response = medical_graph.invoke({"initial_question": state["initial_question"],
                                     "predefined_next_agent": state["predefined_next_agent"],
                                     "predefined_next_question": state["predefined_next_question"],
                                     "sending_agent": state["sending_agent"],
                                     "ongoing_question": state["ongoing_question"],
                                     "recent_response": state["recent_response"],
                                     "current_information": state["current_information"],
                                     "all_information": state["all_information"],
                                     "all_information_index": state["all_information_index"],
                                     "all_information_database": state["all_information_database"],
                                     "current_report": state["current_report"],
                                     "final_summary": state["final_summary"],
                                     "variables": state["variables"],
                                     "responses": state["responses"],
                                     "prompts": state["prompts"],
                                     "answering_agents": state["answering_agents"]})
    state['predefined_next_agent'] = response['predefined_next_agent']
    state['predefined_next_question'] = response['predefined_next_question']
    state['sending_agent'] = response['sending_agent']
    state['recent_response'] = response['recent_response']
    state['final_summary'] = response['final_summary']
    state['responses'] = response['responses']
    state['prompts'] = response['prompts']
    state['variables'] = response['variables']
    return state

# Build node to invoke statistician graph
def StatisticianAgentInvoker(state: TopAgentState):
    response = statistician_graph.invoke({"initial_question": state["initial_question"],
                                        "predefined_next_agent": state["predefined_next_agent"],
                                        "predefined_next_question": state["predefined_next_question"],
                                        "sending_agent": state["sending_agent"],
                                        "ongoing_question": state["ongoing_question"],
                                        "recent_response": state["recent_response"],
                                        "current_information": state["current_information"],
                                        "all_information": state["all_information"],
                                        "all_information_index": state["all_information_index"],
                                        "all_information_database": state["all_information_database"],
                                        "final_summary": state["final_summary"],
                                        "variables": state["variables"],
                                        "responses": state["responses"],
                                        "prompts": state["prompts"],
                                        "answering_agents": state["answering_agents"]})
    state['predefined_next_agent'] = response['predefined_next_agent']
    state['predefined_next_question'] = response['predefined_next_question']
    state['sending_agent'] = response['sending_agent']
    state['recent_response'] = response['recent_response']
    state['final_summary'] = response['final_summary']
    if 'current_report' in response:
        state['current_report'] = response['current_report']
    state['responses'] = response['responses']
    state['prompts'] = response['prompts']
    state['variables'] = response['variables']
    return state

# Build node to invoke trial expert graph
def TrialistInvoker(state: TopAgentState):
    response = trial_graph.invoke({"initial_question": state["initial_question"],
                                        "predefined_next_agent": state["predefined_next_agent"],
                                        "predefined_next_question": state["predefined_next_question"],
                                        "sending_agent": state["sending_agent"],
                                        "ongoing_question": state["ongoing_question"],
                                        "recent_response": state["recent_response"],
                                        "current_information": state["current_information"],
                                        "all_information": state["all_information"],
                                        "all_information_index": state["all_information_index"],
                                        "all_information_database": state["all_information_database"],
                                        "final_summary": state["final_summary"],
                                        "variables": state["variables"],
                                        "responses": state["responses"],
                                        "prompts": state["prompts"],
                                        "answering_agents": state["answering_agents"]})
    state['predefined_next_agent'] = response['predefined_next_agent']
    state['predefined_next_question'] = response['predefined_next_question']
    state['sending_agent'] = response['sending_agent']
    state['recent_response'] = response['recent_response']
    state['final_summary'] = response['final_summary']
    state['responses'] = response['responses']
    state['prompts'] = response['prompts']
    state['variables'] = response['variables']
    return state

def InformaticianAgentInvoker(state: TopAgentState):
    response = informatician_graph.invoke({"initial_question": state["initial_question"],
                                        "predefined_next_agent": state["predefined_next_agent"],
                                        "predefined_next_question": state["predefined_next_question"],
                                        "sending_agent": state["sending_agent"],
                                        "ongoing_question": state["ongoing_question"],
                                        "recent_response": state["recent_response"],
                                        "current_information": state["current_information"],
                                        "all_information": state["all_information"],
                                        "all_information_index": state["all_information_index"],
                                        "all_information_database": state["all_information_database"],
                                        "final_summary": state["final_summary"],
                                        "variables": state["variables"],
                                        "responses": state["responses"],
                                        "prompts": state["prompts"],
                                        "answering_agents": state["answering_agents"]})
    state['predefined_next_agent'] = response['predefined_next_agent']
    state['predefined_next_question'] = response['predefined_next_question']
    state['sending_agent'] = response['sending_agent']
    state['recent_response'] = response['recent_response']
    state['final_summary'] = response['final_summary']
    state['responses'] = response['responses']
    state['prompts'] = response['prompts']
    state['variables'] = response['variables']
    return state

# Build the top level graph
top_members = ["ClinicianAgentInvoker", "StatisticianAgentInvoker", "InformaticianAgentInvoker", "TrialistInvoker"]
top_builder = StateGraph(TopAgentState)
top_builder.add_node(MainSupervisor)
top_builder.add_node(ClinicianAgentInvoker)
top_builder.add_node(StatisticianAgentInvoker)
top_builder.add_node(TrialistInvoker)
top_builder.add_node(InformaticianAgentInvoker)
top_builder.add_edge(START, "MainSupervisor")
for member in top_members:
    top_builder.add_edge(member, "MainSupervisor")
top_builder.add_conditional_edges("MainSupervisor", lambda state: state["next"])
top_graph = top_builder.compile()
display(Image(top_graph.get_graph().draw_mermaid_png()))

In [ ]:
# Define the initial state
# Input
if TRIAL_ID == "NCT04134403":
    initial_input = {"initial_question": "Design a trial emulation with proper protocols and eligibility criteria to determine the effectiveness of hydrocortisone in treating sepsis patients within the ICU.\
    Follow the steps broadly outlined below: \
    1. Get a trial to emulate from the Trialist. \
    2. Build the dataset for the trial emulation with the Informatician. \
    3. Conduct the trial emulation with the Statistician. \
    If there are any issues, the Clinician should be consulted.",
                    "final_summary": "",
                    "current_report": "",
                    "ongoing_question": "",
                    "current_information": "",
                    "recent_response": "",
                    "all_information": "",
                    "all_information_index": None,
                    "all_information_database": None,
                    "predefined_next_agent": "",
                    "predefined_next_question": "",
                    "sending_agent": "",
                    "variables": {'trialid': 'NCT04134403'},
                    "responses": [],
                    "prompts": [],
                    "answering_agents": []}
    
elif TRIAL_ID == "NCT02856698":
    initial_input = {"initial_question": "Determine the effectiveness of treating Acute Pulmonary Edema with midazolam.\
    Follow the steps broadly outlined below: \
    1. Get a trial to emulate from the Trialist. \
    2. Build the dataset for the trial emulation with the Informatician. \
    3. Conduct the trial emulation with the Statistician. \
    If there are any issues, the Clinician should be consulted.",
                    "final_summary": "",
                    "current_report": "",
                    "ongoing_question": "",
                    "current_information": "",
                    "recent_response": "",
                    "all_information": "",
                    "all_information_index": None,
                    "all_information_database": None,
                    "predefined_next_agent": "",
                    "predefined_next_question": "",
                    "sending_agent": "",
                    "variables": {'trialid': 'NCT02856698'},
                    "responses": [],
                    "prompts": [],
                    "answering_agents": []}

elif TRIAL_ID == "NCT06091982":
    initial_input = {"initial_question": "Design a trial emulation to determine what is the effect of renal replacement therapy for severe acute kidney injury.\
    Follow the steps broadly outlined below: \
    Get a trial to emulate from the Trialist. \
    Build the dataset for the trial emulation with the Informatician. \
    Conduct the trial emulation with the Statistician. \
    If there are any issues, the Clinician should be consulted. \
    Do not assume the steps are done.",
                    "final_summary": "",
                    "current_report": "",
                    "ongoing_question": "",
                    "current_information": "",
                    "recent_response": "",
                    "all_information": "",
                    "all_information_index": None,
                    "all_information_database": None,
                    "predefined_next_agent": "",
                    "predefined_next_question": "",
                    "sending_agent": "",
                    "variables": {'trialid': 'NCT06091982'},
                    "responses": [],
                    "prompts": [],
                    "answering_agents": []}

elif TRIAL_ID == "NCT00475852":
    initial_input = {"initial_question": "Design a trial emulation to determine what is the effect of nesiritide for heart failure.\
    Follow the steps broadly outlined below: \
    Get a trial to emulate from the Trialist. \
    Build the dataset for the trial emulation with the Informatician. \
    Conduct the trial emulation with the Statistician. \
    If there are any issues, the Clinician should be consulted. \
    Do not assume the steps are done.",
                    "final_summary": "",
                    "current_report": "",
                    "ongoing_question": "",
                    "current_information": "",
                    "recent_response": "",
                    "all_information": "",
                    "all_information_index": None,
                    "all_information_database": None,
                    "predefined_next_agent": "",
                    "predefined_next_question": "",
                    "sending_agent": "",
                    "variables": {'trialid': 'NCT00475852'},
                    "responses": [],
                    "prompts": [],
                    "answering_agents": []}
    
elif TRIAL_ID == "NCT03872011":
    initial_input = {"initial_question": "Design a trial emulation to determine what is the effect of steroids for septic shock.\
    Follow the steps broadly outlined below: \
    Get a trial to emulate from the Trialist. \
    Build the dataset for the trial emulation with the Informatician. \
    Conduct the trial emulation with the Statistician. \
    If there are any issues, the Clinician should be consulted. \
    Do not assume the steps are done.",
                    "final_summary": "",
                    "current_report": "",
                    "ongoing_question": "",
                    "current_information": "",
                    "recent_response": "",
                    "all_information": "",
                    "all_information_index": None,
                    "all_information_database": None,
                    "predefined_next_agent": "",
                    "predefined_next_question": "",
                    "sending_agent": "",
                    "variables": {'trialid': 'NCT03872011'},
                    "responses": [],
                    "prompts": [],
                    "answering_agents": []}
    

elif TRIAL_ID == "NCT04691505":
    initial_input = {"initial_question": "Design a trial emulation to determine what is the effect of Hydroxychloroquine for dementia.\
    Follow the steps broadly outlined below: \
    Get a trial to emulate from the Trialist. \
    Build the dataset for the trial emulation with the Informatician. \
    Conduct the trial emulation with the Statistician. \
    If there are any issues, the Clinician should be consulted. \
    Do not assume the steps are done.",
                    "final_summary": "",
                    "current_report": "",
                    "ongoing_question": "",
                    "current_information": "",
                    "recent_response": "",
                    "all_information": "",
                    "all_information_index": None,
                    "all_information_database": None,
                    "predefined_next_agent": "",
                    "predefined_next_question": "",
                    "sending_agent": "",
                    "variables": {'trialid': 'NCT04691505', 'dataset': 'insight'},
                    "responses": [],
                    "prompts": [],
                    "answering_agents": []}

else:
    assert False, "Invalid Trial ID"
    
# Thread
thread = {"configurable": {"thread_id": "1"}, "recursion_limit": 100}

# Run the graph until the first interruption
events = []
for event in top_graph.stream(initial_input, thread, stream_mode="values"):
    events.append(event)
print("\n")
print("Final answer: ", event["final_summary"])

In [ ]:
print("Final answer: ", event["final_summary"])

In [ ]:
def summarize_responses(responses):
    summarized_responses = []
    for response in responses:
        summarize_llm_prompt = f'''
        Summarize the following RESPONSE. Keep relevant details about what is being asked. Your summary will be used in a log.

        RESPONSE: {response}

        --------------------------
        
        The output must be in the following format:
        <response>summarized response</response>

        No extra explanations before or after.
        '''

        summarized_response = llm.invoke(summarize_llm_prompt, use_cache=False)
        summarized_responses.append(summarized_response)
    return summarized_responses

summarized_responses = summarize_responses(events[-1]['responses'])

# Conversation Log
for agent, response in  zip(events[-1]['answering_agents'], summarized_responses):
    print(f"Agent: {agent}")
    print("\n")
    print(f"{response}")
    print("\n\n\n")